# Imports and Settings

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse.linalg
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_openml
from scipy.sparse.linalg import eigsh
from graph import Graph
from helpers import get_hermitian_adjacency_matrix, get_adjacency_matrix, get_degree_matrix, get_laplacian_matrix, compute_k_way_estimate, get_normalised_projected_indicator_vectors, degree_correction, k_means_indicator_vectors, compute_rayleigh_quotients, apply_recursive_st_brute_force, gram_schmidt, get_thresholded_correlation_matrix, largest_connected_component, get_normalised_laplacian

In [ ]:
# make text larger
plt.rc('font', size=20)
# set random seed
np.random.seed(9)

<h1 style="font-size: 50px">Undirected Graphs</h1>

# Generating Graphs From Gaussians
We create a plot of 4 gaussian clusters with 100 nodes each.

In [ ]:
X1 = np.random.multivariate_normal(mean = [0,0], cov = [[1,0.5],[0.5,1]], size = 100)
X2 = np.random.multivariate_normal(mean = [0,5], cov = [[1,-0.5],[-0.5,1]], size = 100)
X3 = np.random.multivariate_normal(mean = [8,0], cov = [[1.5,0],[0,1]], size = 100)
X4 = np.random.multivariate_normal(mean = [7,5], cov = [[1,0.2],[0.2,1]], size = 100)
X = np.concatenate((X1, X2, X3, X4))
# plot with each cluster in a different color
plt.scatter(X1[:,0], X1[:,1], color = 'r')
plt.scatter(X2[:,0], X2[:,1], color = 'b')
plt.scatter(X3[:,0], X3[:,1], color = 'g')
plt.scatter(X4[:,0], X4[:,1], color = 'y')

plt.xlabel('x')
plt.ylabel('y')
#plt.title('4 Gaussian Clusters')
plt.savefig('Figures/4GaussianClusters.png')

In [ ]:
# construct a graph from the data points using a threshold
def construct_graph(X, threshold):
    N = X.shape[0]
    A = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i != j:
                dist = np.linalg.norm(X[i] - X[j])
                if dist < threshold:
                    A[i, j] = 1
    return A

A = construct_graph(X, 4)
plt.figure()
G = nx.from_numpy_array(A)
pos = X
# make edges thin and transparent. 
colormap = ['r'] * 100 + ['b'] * 100 + ['g'] * 100 + ['y'] * 100
nx.draw(G, pos = pos, node_size = 10, edge_color = 'grey', width = 0.2, node_color = colormap)
#plt.title('Graph Constructed from 4 Gaussian Clusters \n Threshold = 4')
plt.savefig('Figures/4GaussianClustersGraphThreshold4.png', bbox_inches = 'tight')
plt.close()

In [ ]:
# compute the normalized laplacian

degrees = np.sum(A, axis = 1)
D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
norm_L = np.eye(A.shape[0]) - D_inv_sqrt @ A @ D_inv_sqrt
eigvals, eigvecs = np.linalg.eigh(norm_L)
idx = np.argsort(eigvals)
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

# plot first 10 eigenvalues
fig = plt.figure(figsize=(8, 3))
plt.grid(True)
plt.scatter(range(1,9), eigvals[:8])
plt.xlabel('Index')
plt.xlim(1, 8.6)
plt.ylabel('Eigenvalue')
plt.yticks(np.arange(0, 1.1, 0.2))
plt.yticks(np.arange(0, 1.1, 0.1), minor=True)
#plt.title(r"First 8 Eigenvalues of $\mathcal{L}$")
for i in range(8):
    plt.text(i + 1, eigvals[i], f'{eigvals[i]:.2f}', fontsize=10)
plt.savefig('Figures/4GaussianClusters8Eigenvalues.png')

## Bounds for Geometric Graphs
We take 4 multivariate normally distributed random variables $X_1, X_2,X_3$ and $X_4$ with following distributions:
$X_1 \sim \mathcal{N}((0,0), I)$,
$X_2 \sim \mathcal{N}((0,5), I)$,
$X_3 \sim \mathcal{N}((8,0), I)$,
$X_4 \sim \mathcal{N}((8,5), I)$
We study our bounds for varied number of samples of each distribution.

In [ ]:
rst_bounds = {}
st_bounds = {}
k_way_bounds = {}
true_values = {}
n_clusters = 4
sample_size = 10
for n in np.arange(200,1000,100):
    rst_bounds_temp = []
    st_bounds_temp = []
    k_way_bounds_temp = []
    true_values_temp = []
    print(f"commencing computation for n = {n}")
    for p in range(sample_size):
    
        X1 = np.random.multivariate_normal(mean = [0,0], cov = [[1,0],[0,1]], size = n)
        X2 = np.random.multivariate_normal(mean = [0,5], cov = [[1,0],[0,1]], size = n)
        X3 = np.random.multivariate_normal(mean = [8,0], cov = [[1,0],[0,1]], size = n)
        X4 = np.random.multivariate_normal(mean = [8,5], cov = [[1,0],[0,1]], size = n)
        X = np.concatenate((X1, X2, X3, X4))
        
        A = construct_graph(X, 4)
        degrees = np.sum(A, axis = 1)
        D_sqrt = np.diag(np.sqrt(degrees))
        D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
        norm_L = np.eye(A.shape[0]) - D_inv_sqrt @ A @ D_inv_sqrt
        eigvals, eigvecs = np.linalg.eigh(norm_L)
        idx = np.argsort(eigvals)
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        
        kmeans = KMeans(n_clusters=n_clusters)
        labels = kmeans.fit_predict(eigvecs[:, 0:n_clusters])
        
        #compute exact indicator vectors (from clusters from distributions not k means)
        exact_indicator_vectors = np.zeros((eigvecs.shape[0], n_clusters))
        for i in range(n_clusters):
            exact_indicator_vectors[i*n:(i+1)*n,i] = 1
        
        for i in range(n_clusters):
            exact_indicator_vectors[:, i] = D_sqrt @ exact_indicator_vectors[:, i]
            exact_indicator_vectors[:, i] = exact_indicator_vectors[:, i] / np.linalg.norm(exact_indicator_vectors[:, i])
        
        true_val = np.linalg.norm(eigvecs[:,:n_clusters] - exact_indicator_vectors @ (exact_indicator_vectors.T @ eigvecs[:,:n_clusters]), ord = 'fro')**2
        
        # indicator vectors from k means
        indicator_vectors = np.zeros((eigvecs.shape[0], n_clusters))
        for i in range(n_clusters):
            indicator_vectors[:, i] = (labels == i).astype(int)
        
        
        for i in range(n_clusters):
            indicator_vectors[:, i] = D_sqrt @ indicator_vectors[:, i]
            indicator_vectors[:, i] = indicator_vectors[:, i]
            indicator_vectors[:, i] = indicator_vectors[:, i] / np.linalg.norm(indicator_vectors[:, i])
        
        # project indicator vectors onto the eigenvectors
        beta_K_by_K = (indicator_vectors.T @ eigvecs[:,:n_clusters])
        
        combined_indicator_vectors = indicator_vectors @ beta_K_by_K
        for i in range(n_clusters):
            combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] / np.linalg.norm(
                combined_indicator_vectors[:, i])
            for j in range(i):
                combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] - (
                            combined_indicator_vectors[:, j].T @ combined_indicator_vectors[:,
                                                                 i]) * combined_indicator_vectors[:, j]
            
        for i in range(n_clusters):
            combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] / np.linalg.norm(combined_indicator_vectors[:, i])
            
            
        # compute the rayleigh quotients
        rayleigh_quotients = []
        for i in range(n_clusters):
            indicator = combined_indicator_vectors[:, i]
            val = (indicator.T @ norm_L @ indicator) / (indicator.T @ indicator)
            rayleigh_quotients.append(val)
        
        # sort the rayleigh quotients
        sorted_rayleigh_quotients = np.sort(rayleigh_quotients)
        B_1 = (sorted_rayleigh_quotients[0] + sorted_rayleigh_quotients[1])/eigvals[2]
        B_2 = ((sorted_rayleigh_quotients[2] + sorted_rayleigh_quotients[3]) - 2*eigvals[2] + eigvals[4]*B_1)/(eigvals[4] - eigvals[2])
        
        rst_bound = B_1 + B_2
        st_bound = (np.sum(sorted_rayleigh_quotients))/(eigvals[4])
        rho = compute_k_way_estimate(norm_L, indicator_vectors, 4)
        k_way_bound = 4*rho/eigvals[4]
        
        rst_bounds_temp.append(rst_bound)
        st_bounds_temp.append(st_bound)
        k_way_bounds_temp.append(k_way_bound)
        true_values_temp.append(true_val)
    
    rst_bounds[n] = np.mean(rst_bounds_temp)
    st_bounds[n] = np.mean(st_bounds_temp)
    k_way_bounds[n] = np.mean(k_way_bounds_temp)
    true_values[n] = np.mean(true_values_temp)
    

In [ ]:
bounds_df = pd.DataFrame([rst_bounds, st_bounds, k_way_bounds, true_values]).T
bounds_df.columns = [r'Theorem 4',r'Theorem 1', r'MacGregor & Sun', r'True Value']
bounds_df.to_csv("Data/4GeometricClusters2Pairs.csv")

df_copy = pd.read_csv("Data/4GeometricClusters2Pairs.csv")
df_copy = df_copy.set_index(["Unnamed: 0"])
df_copy.columns = ["Theorem 4", "Theorem 1", "Macgregor & Sun", "True Value"]
(df_copy / 4).plot(marker = 'o', xlabel = 'n', ylabel = 'Bound Value', figsize = (12,10))
plt.xlabel(r'Cluster size $n$', fontsize = 20)
plt.ylabel(r'Error', fontsize = 20)
plt.grid(True)
# make legend larger
plt.legend(fontsize='large', bbox_to_anchor = (1.0,1.05))
plt.savefig('Data/4GeometricClusters2Pairs.png', bbox_inches = "tight")

No we try a similar experiment but rather than driving the means apart we drive the means of the distributions apart. So,
$X_1 \sim \mathcal{N}((0,0), I)$,
$X_2 \sim \mathcal{N}((0,5), I)$,
$X_3 \sim \mathcal{N}((d,0), I)$,
$X_4 \sim \mathcal{N}((d,5), I)$
d is a parameter that is varied and cluster size n is fixed at 100.


In [ ]:
rst_bounds = {}
st_bounds = {}
k_way_bounds = {}
true_values = {}
n_clusters = 4
n = 100
sample_size = 1
for d in np.arange(5, 12, 0.5):
    rst_bounds_temp = []
    st_bounds_temp = []
    k_way_bounds_temp = []
    true_values_temp = []
    print(f"commencing computation for d = {d}")
    for p in range(sample_size):
    
        X1 = np.random.multivariate_normal(mean = [0,0], cov = [[1,0],[0,1]], size = n)
        X2 = np.random.multivariate_normal(mean = [0,5], cov = [[1,0],[0,1]], size = n)
        X3 = np.random.multivariate_normal(mean = [d,0], cov = [[1,0],[0,1]], size = n)
        X4 = np.random.multivariate_normal(mean = [d,5], cov = [[1,0],[0,1]], size = n)
        X = np.concatenate((X1, X2, X3, X4))
        
        A = construct_graph(X, 4)
        degrees = np.sum(A, axis = 1)
        D_sqrt = np.diag(np.sqrt(degrees))
        D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
        norm_L = np.eye(A.shape[0]) - D_inv_sqrt @ A @ D_inv_sqrt
        eigvals, eigvecs = np.linalg.eigh(norm_L)
        idx = np.argsort(eigvals)
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        
        kmeans = KMeans(n_clusters=n_clusters)
        labels = kmeans.fit_predict(eigvecs[:, 0:n_clusters])
        
        #compute exact indicator vectors (from clusters from distributions not k means)
        exact_indicator_vectors = np.zeros((eigvecs.shape[0], n_clusters))
        for i in range(n_clusters):
            exact_indicator_vectors[i*n:(i+1)*n,i] = 1
        
        for i in range(n_clusters):
            exact_indicator_vectors[:, i] = D_sqrt @ exact_indicator_vectors[:, i]
            exact_indicator_vectors[:, i] = exact_indicator_vectors[:, i] / np.linalg.norm(exact_indicator_vectors[:, i])
        
        true_val = np.linalg.norm(eigvecs[:,:n_clusters] - exact_indicator_vectors @ (exact_indicator_vectors.T @ eigvecs[:,:n_clusters]), ord = 'fro')**2
        
        # indicator vectors from k means
        indicator_vectors = np.zeros((eigvecs.shape[0], n_clusters))
        for i in range(n_clusters):
            indicator_vectors[:, i] = (labels == i).astype(int)
        
        
        for i in range(n_clusters):
            indicator_vectors[:, i] = D_sqrt @ indicator_vectors[:, i]
            indicator_vectors[:, i] = indicator_vectors[:, i]
            indicator_vectors[:, i] = indicator_vectors[:, i] / np.linalg.norm(indicator_vectors[:, i])
        
        # project indicator vectors onto the eigenvectors
        beta_K_by_K = (indicator_vectors.T @ eigvecs[:,:n_clusters])
        
        combined_indicator_vectors = indicator_vectors @ beta_K_by_K
        for i in range(n_clusters):
            combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] / np.linalg.norm(
                combined_indicator_vectors[:, i])
            for j in range(i):
                combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] - (
                            combined_indicator_vectors[:, j].T @ combined_indicator_vectors[:,
                                                                 i]) * combined_indicator_vectors[:, j]
            
        for i in range(n_clusters):
            combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] / np.linalg.norm(combined_indicator_vectors[:, i])
            
            
        # compute the rayleigh quotients
        rayleigh_quotients = []
        for i in range(n_clusters):
            indicator = combined_indicator_vectors[:, i]
            val = (indicator.T @ norm_L @ indicator) / (indicator.T @ indicator)
            rayleigh_quotients.append(val)
        
        # sort the rayleigh quotients
        sorted_rayleigh_quotients = np.sort(rayleigh_quotients)
        B_1 = (sorted_rayleigh_quotients[0] + sorted_rayleigh_quotients[1])/eigvals[2]
        B_2 = ((sorted_rayleigh_quotients[2] + sorted_rayleigh_quotients[3]) - 2*eigvals[2] + eigvals[4]*B_1)/(eigvals[4] - eigvals[2])
        
        rst_bound = B_1 + B_2
        st_bound = (np.sum(sorted_rayleigh_quotients))/(eigvals[4])
        rho = compute_k_way_estimate(norm_L, indicator_vectors, 4)
        k_way_bound = 4*rho/eigvals[4]
        
        rst_bounds_temp.append(rst_bound)
        st_bounds_temp.append(st_bound)
        k_way_bounds_temp.append(k_way_bound)
        true_values_temp.append(true_val)
    
    rst_bounds[d] = np.mean(rst_bounds_temp)
    st_bounds[d] = np.mean(st_bounds_temp)
    k_way_bounds[d] = np.mean(k_way_bounds_temp)
    true_values[d] = np.mean(true_values_temp)
    

In [ ]:
bounds_df = pd.DataFrame([rst_bounds, st_bounds, k_way_bounds, true_values]).T
bounds_df.columns = ["Theorem 4", "Theorem 1", "Macgregor & Sun", "True Value"]
bounds_df.to_csv("Data/4GeometricClusters2PairsDriftingApart.csv")

bounds_df = pd.read_csv("Data/4GeometricClusters2PairsDriftingApart.csv", usecols=lambda column: column != "Unnamed: 0")

fig = plt.figure()
(bounds_df / 4).plot(marker='x', markersize=10, figsize=(12, 10))

#plt.legend(bbox_to_anchor=(1,1.05))
plt.xlabel("Distance, d", fontsize=30)
plt.ylabel("Error", fontsize=30)
plt.legend(fontsize=25)
plt.grid(visible=True, which='both', linewidth=1.5)
plt.xticks(fontsize=25)
plt.yticks(np.arange(0, 1.1, 0.1), fontsize=25)
# plt.title(
#     r"Bounds for $\frac{1}{4}\sum_{i=1}^4\|f_i - \hat{g}_i\|^2$ generated from Gaussian mixture model (4 clusters, 2 pairs)" + "\n Distance between pairs of clusters increased",
#     y=1.03)
plt.savefig("Data/BoundsGaussianMixtureModel2PairsDriftingApart.png")

# Bounds for Stochastic Block Models
Our initial choice is an SBM with 4 clusters and where two pairs have a high affinity for each other. We use 
$$
P = \begin{pmatrix}
    0.5 & 0.4 & 0.1 & 0.1 \\
    0.4 & 0.5 & 0.1 & 0.1 \\
    0.1 & 0.1 & 0.5 & 0.4 \\
    0.1 & 0.1 & 0.4 & 0.5 \\
    \end{pmatrix}
$$

In [ ]:
P = np.array([[0.5, 0.4, 0.1, 0.1],
              [0.4, 0.5, 0.1, 0.1],
              [0.1, 0.1, 0.5, 0.4],
              [0.1, 0.1, 0.4, 0.5]])
K = 4

In [ ]:
bounds = {}
sample_size = 10
for n in [200,300,400,500,600,700,800,900]:
    bounds[n] = 0
    for _ in range(sample_size):
        edges = []
        for i in range(K):
            for j in range(i,K):
                prob_existing_edge = P[i,j]
                if i == j:
                    for u in range(n):
                        for v in range(u+1,n):
                            if np.random.rand() <= prob_existing_edge:
                                edges.append((i * n + u, j * n + v))
                                
                else:
                    for u in range(n):
                        for v in range(n):
                            if np.random.rand() <= prob_existing_edge:
                                edges.append((i * n + u, j * n + v))
        
        true_clusters = [list(range(i*n, (i+1)*n)) for i in range(K)]                        
        G = Graph(vertices = list(range(n * K)), edges = edges)
        norm_L = get_laplacian_matrix(G, normalized=True)
        eigvals, eigvecs = np.linalg.eigh(norm_L)
        idx = np.argsort(eigvals)
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        eigvecs = eigvecs[:, 0:K]
        
        D = get_degree_matrix(G)
        degrees = np.diag(D)
        D_sqrt = np.diag(np.sqrt(degrees))
        indicator_vectors = k_means_indicator_vectors(eigvecs, K)
        degree_corrected_indicator_vectors = degree_correction(indicator_vectors, D_sqrt)
        combined_indicator_vectors = get_normalised_projected_indicator_vectors(eigvecs,degree_corrected_indicator_vectors, K)
        rayleigh_quotients = compute_rayleigh_quotients(norm_L, combined_indicator_vectors, K)
        rayleigh_quotients = np.sort(rayleigh_quotients)
        
        true_value = (1/K) * np.linalg.norm(eigvecs - degree_corrected_indicator_vectors @ (degree_corrected_indicator_vectors.T @ eigvecs), ord = 'fro')**2
        general_ST = (1/K) * (np.sum(rayleigh_quotients))/eigvals[K]
        ST_standard = compute_k_way_estimate(norm_L, degree_corrected_indicator_vectors, K)/ eigvals[K]
        min_split_indices, min_split = apply_recursive_st_brute_force(rayleigh_quotients,eigvals, K)
        recursive_ST = (1/K) * min_split
        
        
        
        bounds[n] = bounds[n] + pd.Series({'Recursive ST': recursive_ST,
            'General ST': general_ST,
            'ST Standard': ST_standard,
            'True Value': true_value} )
        
    bounds[n] = bounds[n] / sample_size

In [ ]:
df = pd.DataFrame(bounds).T
columns = ['Recursive ST', 'General ST', r'$\frac{\rho(4)}{\lambda_5}$', 'True Value']
df.columns = columns
#df = df.drop(columns = ['General ST'])
df.to_csv('Data/Bounds4Clusters2PairsRST.csv')

df_copy = pd.read_csv("Data/Bounds4Clusters2PairsRST.csv")
df_copy = df_copy.set_index(["Unnamed: 0"])
df_copy.columns = ["Theorem 4", "Theorem 1", "Macgregor & Sun", "True Value"]
df_copy.plot(marker='x', markersize=10, xlabel = 'n', ylabel = 'Bound Value', figsize = (12,10))

plt.xlabel(r'Cluster size $n$', fontsize=30)
plt.ylabel(r'Error',fontsize=30)
# add ticks
#plt.xticks(np.arange(200,1000,100))
#plt.yticks(np.arange(0,1.2,0.1))
plt.xticks(fontsize=25)
plt.yticks(np.arange(0,1.1,0.1),fontsize=25)
plt.grid(visible=True, which='both', linewidth=1.5)
# make legend larger
plt.legend(fontsize=25, bbox_to_anchor = (1.0,1.05))
plt.savefig('Data/Bounds4Clusters2PairsRST.png', bbox_inches = "tight")

Repeating the experiment for an SBM with 8 clusters with a single pair that have an affinity for each other.

In [ ]:
K = 8
p1 = 0.5
p2 = 0.3
q = 0.05
P = np.ones((8,8))
P = q*P
np.fill_diagonal(P,p1)
P[0,1] = p2
P[1,0] = p2

In [ ]:
bounds = {}
sample_size = 1
for n in [200,300,400,500,600,700,800,900]:
    bounds[n] = 0
    for _ in range(sample_size):
        edges = []
        for i in range(K):
            for j in range(i,K):
                prob_existing_edge = P[i,j]
                if i == j:
                    for u in range(n):
                        for v in range(u+1,n):
                            if np.random.rand() <= prob_existing_edge:
                                edges.append((i * n + u, j * n + v))
                                
                else:
                    for u in range(n):
                        for v in range(n):
                            if np.random.rand() <= prob_existing_edge:
                                edges.append((i * n + u, j * n + v))
        
        true_clusters = [list(range(i*n, (i+1)*n)) for i in range(K)]                        
        G = Graph(vertices = list(range(n * K)), edges = edges)
        norm_L = get_laplacian_matrix(G, normalized=True)
        eigvals, eigvecs = np.linalg.eigh(norm_L)
        idx = np.argsort(eigvals)
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        eigvecs = eigvecs[:, 0:K]
        
        D = get_degree_matrix(G)
        degrees = np.diag(D)
        D_sqrt = np.diag(np.sqrt(degrees))
        indicator_vectors = k_means_indicator_vectors(eigvecs, K)
        degree_corrected_indicator_vectors = degree_correction(indicator_vectors, D_sqrt)
        combined_indicator_vectors = get_normalised_projected_indicator_vectors(eigvecs,degree_corrected_indicator_vectors, K)
        rayleigh_quotients = compute_rayleigh_quotients(norm_L, combined_indicator_vectors, K)
        rayleigh_quotients = np.sort(rayleigh_quotients)
        
        true_value = (1/K) * np.linalg.norm(eigvecs - degree_corrected_indicator_vectors @ (degree_corrected_indicator_vectors.T @ eigvecs), ord = 'fro')**2
        general_ST = (1/K) * (np.sum(rayleigh_quotients))/eigvals[K]
        ST_standard = compute_k_way_estimate(norm_L, degree_corrected_indicator_vectors, K)/ eigvals[K]
        min_split_indices, min_split = apply_recursive_st_brute_force(rayleigh_quotients,eigvals, K)
        recursive_ST = (1/K) * min_split
        
        
        
        bounds[n] = bounds[n] + pd.Series({'Recursive ST': recursive_ST,
            'General ST': general_ST,
            'ST Standard': ST_standard,
            'True Value': true_value} )
        
    bounds[n] = bounds[n] / sample_size

In [ ]:
df = pd.DataFrame(bounds).T

columns = ['Recursive ST', 'General ST', r'$\frac{\rho(8)}{\lambda_9}$', 'True Value']
df.columns = columns
#df = df.drop(columns = ['General ST'])
df.to_csv("Data/Bounds8Clusters1PairRST.csv")
df_copy = pd.read_csv("Data/Bounds8Clusters1PairRST.csv")
df_copy = df_copy.set_index(["Unnamed: 0"])
df_copy.columns = ["Theorem 4", "Theorem 1", "Macgregor & Sun", "True Value"]
df_copy.plot(marker = 'x', markersize=10, xlabel = 'n', ylabel = 'Bound Value', figsize = (12,10))
# plt.title(r'Bounds of $\frac{1}{8}\sum_{i=1}^8\|f_i - \hat{g}_i\|^2$ for SBM with 8 clusters (with one pair)', y=1.03)
plt.xlabel(r'Cluster size $n$', fontsize = 30)
plt.ylabel(r'Error', fontsize = 30)
plt.grid(True, which='both', linewidth=1.5)
# add ticks
plt.xticks(fontsize=25)
plt.yticks(np.arange(0,1.1,0.1), fontsize=25)
# make legend larger
plt.legend(fontsize=25, bbox_to_anchor = (1.0,1.05))
plt.savefig('Data/Bounds8Clusters1PairRST.png', bbox_inches = "tight")

Repeating the experiment for an  SBM with 12 clusters and the following probability matrix:
$$P = \begin{pmatrix}
0.9 & 0.7 & 0.2 & 0.2 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 \\
0.7 & 0.9 & 0.2 & 0.2 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 \\
0.2 & 0.2 & 0.9 & 0.7 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 \\
0.2 & 0.2 & 0.7 & 0.9 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 \\
0.05 & 0.05 & 0.05 & 0.05 & 0.9 & 0.7 & 0.2 & 0.2 & 0.05 & 0.05 & 0.05 & 0.05 \\
0.05 & 0.05 & 0.05 & 0.05 & 0.7 & 0.9 & 0.2 & 0.2 & 0.05 & 0.05 & 0.05 & 0.05 \\
0.05 & 0.05 & 0.05 & 0.05 & 0.2 & 0.2 & 0.9 & 0.7 & 0.05 & 0.05 & 0.05 & 0.05 \\
0.05 & 0.05 & 0.05 & 0.05 & 0.2 & 0.2 & 0.7 & 0.9 & 0.05 & 0.05 & 0.05 & 0.05 \\
0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.9 & 0.7 & 0.2 & 0.2 \\
0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.7 & 0.9 & 0.2 & 0.2 \\
0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.2 & 0.2 & 0.9 & 0.7 \\
0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.2 & 0.2 & 0.7 & 0.9 \\
\end{pmatrix}
$$


In [ ]:
p1, p2, p3, p4 = 0.9, 0.7, 0.2, 0.05
K = 12
P = np.array([
    [p1, p2, p3, p3, p4, p4, p4, p4, p4, p4, p4, p4],
    [p2, p1, p3, p3, p4, p4, p4, p4, p4, p4, p4, p4],
    [p3, p3, p1, p2, p4, p4, p4, p4, p4, p4, p4, p4],
    [p3, p3, p2, p1, p4, p4, p4, p4, p4, p4, p4, p4],
    [p4, p4, p4, p4, p1, p2, p3, p3, p4, p4, p4, p4],
    [p4, p4, p4, p4, p2, p1, p3, p3, p4, p4, p4, p4],
    [p4, p4, p4, p4, p3, p3, p1, p2, p4, p4, p4, p4],
    [p4, p4, p4, p4, p3, p3, p2, p1, p4, p4, p4, p4],
    [p4, p4, p4, p4, p4, p4, p4, p4, p1, p2, p3, p3],
    [p4, p4, p4, p4, p4, p4, p4, p4, p2, p1, p3, p3],
    [p4, p4, p4, p4, p4, p4, p4, p4, p3, p3, p1, p2],
    [p4, p4, p4, p4, p4, p4, p4, p4, p3, p3, p2, p1]
])

In [ ]:
import scipy
bounds = {}
sample_size = 5
for n in [200,300,400,500,600,700]:
    bounds[n] = 0
    for _ in range(sample_size):
        edges = []
        for i in range(K):
            for j in range(i,K):
                prob_existing_edge = P[i,j]
                if i == j:
                    for u in range(n):
                        for v in range(u+1,n):
                            if np.random.rand() <= prob_existing_edge:
                                edges.append((i * n + u, j * n + v))
                                
                else:
                    for u in range(n):
                        for v in range(n):
                            if np.random.rand() <= prob_existing_edge:
                                edges.append((i * n + u, j * n + v))
        
        true_clusters = [list(range(i*n, (i+1)*n)) for i in range(K)]                        
        G = Graph(vertices = list(range(n * K)), edges = edges)
        norm_L = get_laplacian_matrix(G, normalized=True)
        eigvals, eigvecs = scipy.sparse.linalg.eigsh(norm_L,15, which='SM')
        idx = np.argsort(eigvals)
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        eigvecs = eigvecs[:, 0:K]
        
        D = get_degree_matrix(G)
        degrees = np.diag(D)
        D_sqrt = np.diag(np.sqrt(degrees))
        indicator_vectors = k_means_indicator_vectors(eigvecs, K)
        degree_corrected_indicator_vectors = degree_correction(indicator_vectors, D_sqrt)
        combined_indicator_vectors = get_normalised_projected_indicator_vectors(eigvecs,degree_corrected_indicator_vectors, K)
        rayleigh_quotients = compute_rayleigh_quotients(norm_L, combined_indicator_vectors, K)
        rayleigh_quotients = np.sort(rayleigh_quotients)
        
        true_value = (1/K) * np.linalg.norm(eigvecs - degree_corrected_indicator_vectors @ (degree_corrected_indicator_vectors.T @ eigvecs), ord = 'fro')**2
        general_ST = (1/K) * (np.sum(rayleigh_quotients))/eigvals[K]
        ST_standard = compute_k_way_estimate(norm_L, degree_corrected_indicator_vectors, K)/ eigvals[K]
        min_split_indices, min_split = apply_recursive_st_brute_force(rayleigh_quotients,eigvals, K)
        recursive_ST = (1/K) * min_split
        
        
        
        bounds[n] = bounds[n] + pd.Series({'Recursive ST': recursive_ST,
            'General ST': general_ST,
            'ST Standard': ST_standard,
            'True Value': true_value} )
        
    bounds[n] = bounds[n] / sample_size

In [ ]:
df = pd.DataFrame(bounds).T

columns = ['Recursive ST', 'General ST', r'$\frac{\rho(8)}{\lambda_9}$', 'True Value']
df.columns = columns
#df = df.drop(columns = ['General ST'])
df.to_csv("Data/Bounds12Clusters3QuartetsRST.csv")
df_copy = pd.read_csv("Data/Bounds12Clusters3QuartetsRST.csv")
df_copy = df_copy.set_index(["Unnamed: 0"])
df_copy.columns = ["Theorem 4", "Theorem 1", "Macgregor & Sun", "True Value"]
df_copy.plot(marker='x', markersize=10, xlabel='n', ylabel='Bound Value', figsize=(12, 10))
# plt.title(r'Bounds of $\frac{1}{8}\sum_{i=1}^8\|f_i - \hat{g}_i\|^2$ for SBM with 8 clusters (with one pair)', y=1.03)
plt.xlabel(r'Cluster size $n$', fontsize=30)
plt.ylabel(r'Error', fontsize=30)
plt.grid(True, which='both', linewidth=1.5)
# add ticks
plt.xticks(fontsize=25)
plt.yticks(np.arange(0, 1.1, 0.1), fontsize=25)
# make legend larger
plt.legend(fontsize=25, bbox_to_anchor=(1.0, 1.05))
plt.savefig('Data/Bounds12Clusters3QuartetsRST.png', bbox_inches="tight")

# Real-World Networks
In this section we produce our bounds for a collection of real-world networks.

## MNIST Dataset
Sourced from openml. We sample 500 images each of digits 0,1,2,3 and 4

In [ ]:
dataset_results = {}

In [ ]:
# Load the MNIST dataset using OpenML
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
x_data, y_data = mnist.data, mnist.target.astype(int)

# Select 3 digits (e.g., 0, 1, and 2) and restrict to 200 samples each
digits = [0, 1, 2, 3, 4]
samples_per_digit = 500
selected_samples = []
selected_labels = []

# taking a sample of each digit
for digit in digits:
    indices = np.where(y_data == digit)[0][:samples_per_digit]
    selected_samples.append(x_data[indices])
    selected_labels.extend(y_data[indices])

# Combine the samples into a single dataset
selected_samples = np.vstack(selected_samples) / 255.0  # Normalize pixel values
selected_labels = np.array(selected_labels)

In [ ]:
# Create a graph using the correlation matrix
threshold = 0.7  # Define a threshold for edge creation
adjacency_matrix = get_thresholded_correlation_matrix(selected_samples, threshold)
adjacency_matrix_largest_cc, largest_cc = largest_connected_component(adjacency_matrix) 
normalized_laplacian = get_normalised_laplacian(adjacency_matrix_largest_cc)

# compute first K eigenvectors of the normalized Laplacian
K=5
normalized_L_eigenvalues, normalized_L_eigenvectors = np.linalg.eigh(normalized_laplacian)
idx = normalized_L_eigenvalues.argsort()
normalized_L_eigenvalues = normalized_L_eigenvalues[idx]
normalized_L_eigenvectors = normalized_L_eigenvectors[:, idx]

indicator_vectors = k_means_indicator_vectors(normalized_L_eigenvectors[:, 0:K], K)
degrees = np.sum(adjacency_matrix_largest_cc, axis = 0)
D_sqrt = np.diag(np.sqrt(degrees))

indicator_vectors = degree_correction(indicator_vectors, D_sqrt)
beta_K_by_K = indicator_vectors.T @ normalized_L_eigenvectors[:, 0:K]
combined_indicator_vectors = indicator_vectors @ beta_K_by_K
for i in range(K):
    combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] / np.linalg.norm(
        combined_indicator_vectors[:, i])
    for j in range(i):
        combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] - (
                    combined_indicator_vectors[:, j].T @ combined_indicator_vectors[:,
                                                         i]) * combined_indicator_vectors[:, j]

rayleigh_quotients = compute_rayleigh_quotients(normalized_laplacian, combined_indicator_vectors, K)
rayleigh_quotients = np.sort(rayleigh_quotients)

In [ ]:
true_val_matrix_combined = normalized_L_eigenvectors[:, :K] - indicator_vectors @ (
            indicator_vectors.T @ normalized_L_eigenvectors[:, :K])
true_val_combined = (1 / K) * np.linalg.norm(true_val_matrix_combined)
split, v = apply_recursive_st_brute_force(rayleigh_quotients, normalized_L_eigenvalues[0:K+1], K)

mnist_dataset_results = {}
print("K: ", K)
mnist_dataset_results["K"] = K
print("Number of vertices N: ", len(adjacency_matrix_largest_cc))
mnist_dataset_results["N"] = len(adjacency_matrix_largest_cc)
print("Number of edges M: ", np.sum(np.sum(adjacency_matrix_largest_cc, axis=1), axis=0))
mnist_dataset_results["M"] = np.sum(np.sum(adjacency_matrix_largest_cc, axis=1), axis=0)
print("True value: ", true_val_combined)
mnist_dataset_results["True Value"] = true_val_combined
print("M&S ST: ",
      max(np.diag(indicator_vectors.T @ normalized_laplacian @ indicator_vectors)) / normalized_L_eigenvalues[K])
mnist_dataset_results["M&S ST"] = max(np.diag(indicator_vectors.T @ normalized_laplacian @ indicator_vectors)) / \
                                  normalized_L_eigenvalues[K]
print("General ST: ", np.sum(rayleigh_quotients) / (K * normalized_L_eigenvalues[K]))
mnist_dataset_results["General ST"] = np.sum(rayleigh_quotients) / (K * normalized_L_eigenvalues[K])
print("Recursive ST: ", v / K)
mnist_dataset_results["Recursive ST"] = v / K

dataset_results["MNIST"] = mnist_dataset_results

# Fashion MNIST

In [ ]:
# Load the MNIST dataset using OpenML
mnist = fetch_openml('Fashion-MNIST', version=1, as_frame=False)
x_data, y_data = mnist.data, mnist.target.astype(int)

# Select 3 digits (e.g., 0, 1, and 2) and restrict to 200 samples each
digits = [0, 1, 2]
samples_per_digit = 1000
selected_samples = []
selected_labels = []

# taking a sample of each digit
for digit in digits:
    indices = np.where(y_data == digit)[0][:samples_per_digit]
    selected_samples.append(x_data[indices])
    selected_labels.extend(y_data[indices])

# Combine the samples into a single dataset
selected_samples = np.vstack(selected_samples) / 255.0  # Normalize pixel values
selected_labels = np.array(selected_labels)

# Create a graph using the correlation matrix
threshold = 0.7  # Define a threshold for edge creation
adjacency_matrix = get_thresholded_correlation_matrix(selected_samples, threshold)
adjacency_matrix_largest_cc, largest_cc = largest_connected_component(adjacency_matrix) 


This dataset had some vertices that need to be removed.

In [ ]:
vertices_to_remove = [318,319,320]
adjacency_matrix_largest_cc = np.delete(adjacency_matrix_largest_cc, vertices_to_remove, axis=0)
adjacency_matrix_largest_cc = np.delete(adjacency_matrix_largest_cc, vertices_to_remove, axis=1)
normalized_laplacian = get_normalised_laplacian(adjacency_matrix_largest_cc)

In [ ]:
# compute first K eigenvectors of the normalized Laplacian
K=6
normalized_L_eigenvalues, normalized_L_eigenvectors = np.linalg.eigh(normalized_laplacian)
idx = normalized_L_eigenvalues.argsort()
normalized_L_eigenvalues = normalized_L_eigenvalues[idx]
normalized_L_eigenvectors = normalized_L_eigenvectors[:, idx]

indicator_vectors = k_means_indicator_vectors(normalized_L_eigenvectors[:, 0:K], K)
degrees = np.sum(adjacency_matrix_largest_cc, axis = 0)
D_sqrt = np.diag(np.sqrt(degrees))

indicator_vectors = degree_correction(indicator_vectors, D_sqrt)
beta_K_by_K = indicator_vectors.T @ normalized_L_eigenvectors[:, 0:K]
combined_indicator_vectors = indicator_vectors @ beta_K_by_K
for i in range(K):
    combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] / np.linalg.norm(
        combined_indicator_vectors[:, i])
    for j in range(i):
        combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] - (
                    combined_indicator_vectors[:, j].T @ combined_indicator_vectors[:,
                                                         i]) * combined_indicator_vectors[:, j]

rayleigh_quotients = compute_rayleigh_quotients(normalized_laplacian, combined_indicator_vectors, K)
rayleigh_quotients = np.sort(rayleigh_quotients)

In [ ]:
split, v = apply_recursive_st_brute_force(rayleigh_quotients, normalized_L_eigenvalues[0:7], K)
true_val_matrix_combined = normalized_L_eigenvectors[:, :K] - indicator_vectors @ (
        indicator_vectors.T @ normalized_L_eigenvectors[:, :K])
true_val_combined = (1 / K) * np.linalg.norm(true_val_matrix_combined)

fashion_mnist_dataset_results = {}
print("K: ", K)
fashion_mnist_dataset_results["K"] = K
print("Number of vertices N: ", len(adjacency_matrix_largest_cc))
fashion_mnist_dataset_results["N"] = len(adjacency_matrix_largest_cc)
print("Number of edges M: ", np.sum(np.sum(adjacency_matrix_largest_cc, axis=1), axis=0))
fashion_mnist_dataset_results["M"] = np.sum(np.sum(adjacency_matrix_largest_cc, axis=1), axis=0)
print("True value: ", true_val_combined)
fashion_mnist_dataset_results["True Value"] = true_val_combined
print("M&S ST: ",
      max(np.diag(indicator_vectors.T @ normalized_laplacian @ indicator_vectors)) / normalized_L_eigenvalues[K])
fashion_mnist_dataset_results["M&S ST"] = max(np.diag(indicator_vectors.T @ normalized_laplacian @ indicator_vectors)) / \
                                          normalized_L_eigenvalues[K]
print("General ST: ", np.sum(rayleigh_quotients) / (K * normalized_L_eigenvalues[K]))
fashion_mnist_dataset_results["General ST"] = np.sum(rayleigh_quotients) / (K * normalized_L_eigenvalues[K])
print("Recursive ST: ", np.sum(v) / K)
fashion_mnist_dataset_results["Recursive ST"] = np.sum(v) / K

dataset_results["Fashion MNIST"] = fashion_mnist_dataset_results

## Air Quality Dataset
Pulled from: https://www.kaggle.com/datasets/mujtabamatin/air-quality-and-pollution-assessment

In [ ]:
pollution_df = pd.read_csv("Data/updated_pollution_dataset.csv")
pollution_df = pollution_df.sort_values(by='Air Quality').reset_index(drop=True)

# Separate targets and features
pollution_df_targets = pollution_df['Air Quality']
pollution_df_features = pollution_df.drop(columns=['Air Quality'])

# Normalize features
pollution_df_features_normalized = (pollution_df_features - pollution_df_features.mean()) / pollution_df_features.std()

# Calculate covariance matrix
pollution_df_features_cov = np.cov(pollution_df_features_normalized)

# Threshold covariance matrix
pollution_df_features_cov_thresholded = pollution_df_features_cov.copy()
pollution_df_features_cov_thresholded[pollution_df_features_cov_thresholded < 0.4] = 0
pollution_df_features_cov_thresholded[pollution_df_features_cov_thresholded >= 0.4] = 1

Removing disconnected components

In [ ]:
# Remove zero rows and columns
non_zero_indices = np.where(~(pollution_df_features_cov_thresholded == 0).all(axis=1))[0]
pollution_df_features_cov_thresholded = pollution_df_features_cov_thresholded[non_zero_indices][:, non_zero_indices]

# Adjacency matrix
adjacency_matrix = pollution_df_features_cov_thresholded.copy()

In [ ]:
normalized_laplacian = get_normalised_laplacian(adjacency_matrix)

# compute first K eigenvectors of the normalized Laplacian
K = 3
normalized_L_eigenvalues, normalized_L_eigenvectors = np.linalg.eigh(normalized_laplacian)
idx = normalized_L_eigenvalues.argsort()
normalized_L_eigenvalues = normalized_L_eigenvalues[idx]
normalized_L_eigenvectors = normalized_L_eigenvectors[:, idx]

indicator_vectors = k_means_indicator_vectors(normalized_L_eigenvectors[:, 0:K], K)
degrees = np.sum(adjacency_matrix, axis=0)
D_sqrt = np.diag(np.sqrt(degrees))

indicator_vectors = degree_correction(indicator_vectors, D_sqrt)
beta_K_by_K = indicator_vectors.T @ normalized_L_eigenvectors[:, 0:K]
combined_indicator_vectors = indicator_vectors @ beta_K_by_K
for i in range(K):
    combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] / np.linalg.norm(
        combined_indicator_vectors[:, i])
    for j in range(i):
        combined_indicator_vectors[:, i] = combined_indicator_vectors[:, i] - (
                combined_indicator_vectors[:, j].T @ combined_indicator_vectors[:,
                                                     i]) * combined_indicator_vectors[:, j]

rayleigh_quotients = compute_rayleigh_quotients(normalized_laplacian, combined_indicator_vectors, K)
rayleigh_quotients = np.sort(rayleigh_quotients)

In [ ]:
true_val_matrix_combined = normalized_L_eigenvectors[:, :K] - indicator_vectors @ (
        indicator_vectors.T @ normalized_L_eigenvectors[:, :K])
true_val_combined = (1 / K) * np.linalg.norm(true_val_matrix_combined)

rayleigh_quotients = np.round(rayleigh_quotients, 6)
normalized_L_eigenvalues = np.round(normalized_L_eigenvalues, 6)


split, v = apply_recursive_st_brute_force(rayleigh_quotients, normalized_L_eigenvalues[0:K+1], K)


air_quality_dataset_results = {}
print("K: ", K)
air_quality_dataset_results["K"] = K
print("Number of vertices N: ", len(adjacency_matrix))
air_quality_dataset_results["N"] = len(adjacency_matrix)
print("Number of edges M: ", np.sum(np.sum(adjacency_matrix, axis=1), axis=0))
air_quality_dataset_results["M"] = np.sum(np.sum(adjacency_matrix, axis=1), axis=0)
print("True value: ", true_val_combined)
air_quality_dataset_results["True Value"] = true_val_combined
print("M&S ST: ", max(np.diag(indicator_vectors.T @ normalized_laplacian @ indicator_vectors))/normalized_L_eigenvalues[K])
air_quality_dataset_results["M&S ST"] = max(np.diag(indicator_vectors.T @ normalized_laplacian @ indicator_vectors))/normalized_L_eigenvalues[K]
print("General ST: ", np.sum(rayleigh_quotients)/(K * normalized_L_eigenvalues[K]))
air_quality_dataset_results["General ST"] = np.sum(rayleigh_quotients)/(K * normalized_L_eigenvalues[K])
print("Recursive ST: ", v/K)
air_quality_dataset_results["Recursive ST"] = v/K

dataset_results["Air Quality"] = air_quality_dataset_results

## Twitch
From SNAP.

In [ ]:
# Step 1: Define the file path
edge_list_path = "Data/twitch_gamers/large_twitch_edges.csv"
df = pd.read_csv(edge_list_path)
df.to_csv("Data/twitch_gamers/twitch_cleaned_edges.csv", index=False, header=False)
# Step 2: Load the Deezer graph
# Assuming the edge list is in CSV format with two columns: source and target
G = nx.read_edgelist("Data/twitch_gamers/twitch_cleaned_edges.csv", delimiter=",", nodetype=int, data=False)

laplacian_sparse = nx.normalized_laplacian_matrix(G).astype(np.float64)

degrees = dict(G.degree())
# Compute the square root of the degrees
inv_sqrt_degrees = np.array([1 / np.sqrt(degrees[node]) if degrees[node] != 0 else 0 for node in G.nodes()])

# Step 4: Compute the smallest 6 eigenvalues of the Laplacian (for example)
num_eigenvalues = 20
eigenvalues, eigenvectors = eigsh(laplacian_sparse, k=num_eigenvalues, which="SM")

In [ ]:
K = 2
# Perform K-means clustering on the eigenvectors
kmeans = KMeans(n_clusters=K, n_init=10, random_state=42)
scaled_eigenvectors = inv_sqrt_degrees[:, np.newaxis] * eigenvectors

clusters = kmeans.fit_predict(scaled_eigenvectors[:, :K])

degrees = dict(G.degree())
# Compute the square root of the degrees
sqrt_degrees = np.array([np.sqrt(degrees[node]) for node in G.nodes()])

# Compute Rayleigh quotients for each cluster
rayleigh_quotients = []
indicator_vectors = np.zeros(eigenvectors.shape)

for cluster_idx in range(K):
    # Create indicator vector for the cluster
    indicator_vector = (clusters == cluster_idx).astype(float) * sqrt_degrees
    indicator_vector = indicator_vector / np.linalg.norm(indicator_vector, 2)
    indicator_vectors[:, cluster_idx] = indicator_vector
    # Compute Rayleigh quotient
    numerator = indicator_vector.T @ laplacian_sparse @ indicator_vector
    denominator = indicator_vector.T @ indicator_vector
    rayleigh_quotients.append(numerator / denominator)

rayleigh_quotients = sorted(rayleigh_quotients)
beta_K_by_K = indicator_vectors.T @ scaled_eigenvectors[:, 0:K]
combined_indicator_vectors = indicator_vectors @ beta_K_by_K
combined_indicator_vectors = gram_schmidt(combined_indicator_vectors)
combined_rayleigh_quotients = []
for i in range(K):
    numerator = combined_indicator_vectors[:, i].T @ laplacian_sparse @ combined_indicator_vectors[:, i]
    denominator = combined_indicator_vectors[:, i].T @ combined_indicator_vectors[:, i]
    combined_rayleigh_quotients.append(numerator / denominator)

combined_rayleigh_quotients = sorted(combined_rayleigh_quotients)

In [ ]:
true_val_matrix_combined = eigenvectors[:, :K] - indicator_vectors @ (
        indicator_vectors.T @ eigenvectors[:, :K])
true_val_combined = (1 / K) * np.linalg.norm(true_val_matrix_combined)

rayleigh_quotients = np.round(rayleigh_quotients, 6)
normalized_L_eigenvalues = np.round(eigenvalues, 6)

#b, v = apply_recursive_st_search(rayleigh_quotients[0:4], normalized_L_eigenvalues[0:4], 0, error = 0, bound_indices = [], values = [])
split, v = apply_recursive_st_brute_force(rayleigh_quotients, eigenvalues[0:K+1], K)


twitch_dataset_results = {}
print("K: ", K)
twitch_dataset_results["K"] = K
print("Number of vertices N: ", len(eigenvectors))
twitch_dataset_results["N"] = len(eigenvectors)
print("Number of edges M: ", np.sum(list(degrees.values())) / 2)
twitch_dataset_results["M"] = np.sum(list(degrees.values())) / 2
print("True value: ", true_val_combined)
twitch_dataset_results["True Value"] = true_val_combined
print("M&S ST: ", max(np.diag(indicator_vectors.T @ laplacian_sparse @ indicator_vectors))/eigenvalues[K])
twitch_dataset_results["M&S ST"] = max(np.diag(indicator_vectors.T @ laplacian_sparse @ indicator_vectors))/eigenvalues[K]
print("General ST: ", np.sum(rayleigh_quotients)/(K * eigenvalues[K]))
twitch_dataset_results["General ST"] = np.sum(rayleigh_quotients)/(K * eigenvalues[K])
print("Recursive ST: ", v/K)
twitch_dataset_results["Recursive ST"] = v/K

dataset_results["Twitch"] = twitch_dataset_results

## Last Fm Asia

In [ ]:
# Step 1: Define the file path
edge_list_path = "Data/lasftm_asia/lastfm_asia_edges.csv"
df = pd.read_csv(edge_list_path)
df.to_csv("Data/lasftm_asia/lastfm_asia_edges.csv", index=False, header=False)
# Step 2: Load the Deezer graph
# Assuming the edge list is in CSV format with two columns: source and target
G = nx.read_edgelist("Data/lasftm_asia/lastfm_asia_edges.csv", delimiter=",", nodetype=int, data=False)

laplacian_sparse = nx.normalized_laplacian_matrix(G).astype(np.float64)

degrees = dict(G.degree())
# Compute the square root of the degrees
inv_sqrt_degrees = np.array([1 / np.sqrt(degrees[node]) if degrees[node] != 0 else 0 for node in G.nodes()])

# Step 4: Compute the smallest 6 eigenvalues of the Laplacian (for example)
num_eigenvalues = 20
eigenvalues, eigenvectors = eigsh(laplacian_sparse, k=num_eigenvalues, which="SM")


In [ ]:
K = 2
# Perform K-means clustering on the eigenvectors
kmeans = KMeans(n_clusters=K, n_init=10, random_state=42)
scaled_eigenvectors = inv_sqrt_degrees[:, np.newaxis] * eigenvectors

clusters = kmeans.fit_predict(scaled_eigenvectors[:, :K])

degrees = dict(G.degree())
# Compute the square root of the degrees
sqrt_degrees = np.array([np.sqrt(degrees[node]) for node in G.nodes()])

# Compute Rayleigh quotients for each cluster
rayleigh_quotients = []
indicator_vectors = np.zeros(eigenvectors.shape)

for cluster_idx in range(K):
    # Create indicator vector for the cluster
    indicator_vector = (clusters == cluster_idx).astype(float) * sqrt_degrees
    indicator_vector = indicator_vector / np.linalg.norm(indicator_vector, 2)
    indicator_vectors[:, cluster_idx] = indicator_vector
    # Compute Rayleigh quotient
    numerator = indicator_vector.T @ laplacian_sparse @ indicator_vector
    denominator = indicator_vector.T @ indicator_vector
    rayleigh_quotients.append(numerator / denominator)

rayleigh_quotients = sorted(rayleigh_quotients)
beta_K_by_K = indicator_vectors.T @ scaled_eigenvectors[:, 0:K]
combined_indicator_vectors = indicator_vectors @ beta_K_by_K
combined_indicator_vectors = gram_schmidt(combined_indicator_vectors)
combined_rayleigh_quotients = []
for i in range(K):
    numerator = combined_indicator_vectors[:, i].T @ laplacian_sparse @ combined_indicator_vectors[:, i]
    denominator = combined_indicator_vectors[:, i].T @ combined_indicator_vectors[:, i]
    combined_rayleigh_quotients.append(numerator / denominator)

combined_rayleigh_quotients = sorted(combined_rayleigh_quotients)

In [ ]:
lastfm_dataset_results = {}
true_val_matrix_combined = eigenvectors[:, :K] - indicator_vectors @ (
        indicator_vectors.T @ eigenvectors[:, :K])
true_val_combined = (1 / K) * np.linalg.norm(true_val_matrix_combined)

rayleigh_quotients = np.round(rayleigh_quotients, 6)
normalized_L_eigenvalues = np.round(eigenvalues, 6)

#b, v = apply_recursive_st_search(rayleigh_quotients[0:4], normalized_L_eigenvalues[0:4], 0, error = 0, bound_indices = [], values = [])
split, v = apply_recursive_st_brute_force(rayleigh_quotients, eigenvalues[0:K+1], K)


lastfm_dataset_results = {}
print("K: ", K)
lastfm_dataset_results["K"] = K
print("Number of vertices N: ", len(eigenvectors))
lastfm_dataset_results["N"] = len(eigenvectors)
print("Number of edges M: ", np.sum(list(degrees.values())) / 2)
lastfm_dataset_results["M"] = np.sum(list(degrees.values())) / 2
print("True value: ", true_val_combined)
lastfm_dataset_results["True Value"] = true_val_combined
print("M&S ST: ", max(np.diag(indicator_vectors.T @ laplacian_sparse @ indicator_vectors))/eigenvalues[K])
lastfm_dataset_results["M&S ST"] = max(np.diag(indicator_vectors.T @ laplacian_sparse @ indicator_vectors))/eigenvalues[K]
print("General ST: ", np.sum(rayleigh_quotients)/(K * eigenvalues[K]))
lastfm_dataset_results["General ST"] = np.sum(rayleigh_quotients)/(K * eigenvalues[K])
print("Recursive ST: ", v/K)
lastfm_dataset_results["Recursive ST"] = v/K

dataset_results["LastFM"] = lastfm_dataset_results

## Gemsec Facebook - Athletes

In [ ]:
# Step 1: Define the file path
edge_list_path = "Data/facebook_clean_data/athletes_edges.csv"
df = pd.read_csv(edge_list_path)
df.to_csv("Data/facebook_clean_data/athletes_edges.csv", index=False, header=False)
# Step 2: Load the Deezer graph
# Assuming the edge list is in CSV format with two columns: source and target
G = nx.read_edgelist("Data/facebook_clean_data/athletes_edges.csv", delimiter=",", nodetype=int, data=False)

laplacian_sparse = nx.normalized_laplacian_matrix(G).astype(np.float64)

degrees = dict(G.degree())
# Compute the square root of the degrees
inv_sqrt_degrees = np.array([1 / np.sqrt(degrees[node]) if degrees[node] != 0 else 0 for node in G.nodes()])

# Step 4: Compute the smallest 6 eigenvalues of the Laplacian (for example)
num_eigenvalues = 20
eigenvalues, eigenvectors = eigsh(laplacian_sparse, k=num_eigenvalues, which="SM")


In [ ]:
K = 3
# Perform K-means clustering on the eigenvectors
kmeans = KMeans(n_clusters=K, n_init=10, random_state=42)
scaled_eigenvectors = inv_sqrt_degrees[:, np.newaxis] * eigenvectors

clusters = kmeans.fit_predict(scaled_eigenvectors[:, :K])

degrees = dict(G.degree())
# Compute the square root of the degrees
sqrt_degrees = np.array([np.sqrt(degrees[node]) for node in G.nodes()])

# Compute Rayleigh quotients for each cluster
rayleigh_quotients = []
indicator_vectors = np.zeros(eigenvectors.shape)

for cluster_idx in range(K):
    # Create indicator vector for the cluster
    indicator_vector = (clusters == cluster_idx).astype(float) * sqrt_degrees
    indicator_vector = indicator_vector / np.linalg.norm(indicator_vector, 2)
    indicator_vectors[:, cluster_idx] = indicator_vector
    # Compute Rayleigh quotient
    numerator = indicator_vector.T @ laplacian_sparse @ indicator_vector
    denominator = indicator_vector.T @ indicator_vector
    rayleigh_quotients.append(numerator / denominator)

rayleigh_quotients = sorted(rayleigh_quotients)
beta_K_by_K = indicator_vectors.T @ scaled_eigenvectors[:, 0:K]
combined_indicator_vectors = indicator_vectors @ beta_K_by_K
combined_indicator_vectors = gram_schmidt(combined_indicator_vectors)

combined_rayleigh_quotients = []
for i in range(K):
    numerator = combined_indicator_vectors[:, i].T @ laplacian_sparse @ combined_indicator_vectors[:, i]
    denominator = combined_indicator_vectors[:, i].T @ combined_indicator_vectors[:, i]
    combined_rayleigh_quotients.append(numerator / denominator)

combined_rayleigh_quotients = sorted(combined_rayleigh_quotients)

In [ ]:
facebook_athletes_dataset_results = {}
true_val_matrix_combined = eigenvectors[:, :K] - indicator_vectors @ (
        indicator_vectors.T @ eigenvectors[:, :K])
true_val_combined = (1 / K) * np.linalg.norm(true_val_matrix_combined)

rayleigh_quotients = np.round(rayleigh_quotients, 6)
normalized_L_eigenvalues = np.round(eigenvalues, 6)

#b, v = apply_recursive_st_search(rayleigh_quotients[0:4], normalized_L_eigenvalues[0:4], 0, error = 0, bound_indices = [], values = [])
split, v = apply_recursive_st_brute_force(rayleigh_quotients, eigenvalues[0:K+1], K)


facebook_athletes_dataset_results = {}
print("K: ", K)
facebook_athletes_dataset_results["K"] = K
print("Number of vertices N: ", len(eigenvectors))
facebook_athletes_dataset_results["N"] = len(eigenvectors)
print("Number of edges M: ", np.sum(list(degrees.values())) / 2)
facebook_athletes_dataset_results["M"] = np.sum(list(degrees.values())) / 2
print("True value: ", true_val_combined)
facebook_athletes_dataset_results["True Value"] = true_val_combined
print("M&S ST: ", max(np.diag(indicator_vectors.T @ laplacian_sparse @ indicator_vectors))/eigenvalues[K])
facebook_athletes_dataset_results["M&S ST"] = max(np.diag(indicator_vectors.T @ laplacian_sparse @ indicator_vectors))/eigenvalues[K]
print("General ST: ", np.sum(rayleigh_quotients)/(K * eigenvalues[K]))
facebook_athletes_dataset_results["General ST"] = np.sum(rayleigh_quotients)/(K * eigenvalues[K])
print("Recursive ST: ", v/K)
facebook_athletes_dataset_results["Recursive ST"] = v/K

dataset_results["Facebook (Athletes)"] = facebook_athletes_dataset_results

## Collaborations CA-CondMat

In [ ]:

# Step 1: Define the file path
# Define the file path for the input and output
input_file = "Data/ca-CondMat.txt/CA-CondMat.txt"  # Replace with your actual file path
output_file = "Data/ca-CondMat.txt/edge_list.txt"

# Initialize a set to store unique edges (to ensure undirected representation)
edge_set = set()

# Open the input file and process each line
with open(input_file, "r") as file:
    for line in file:
        # Skip comment lines starting with '#'
        if line.startswith("#"):
            continue

        # Split the line into nodes
        parts = line.strip().split()
        if len(parts) == 2:
            node1, node2 = parts

            # Add the edge in a sorted order to avoid duplicates (undirected graph)
            edge = tuple(sorted((int(node1), int(node2))))
            edge_set.add(edge)

# Write the unique edges to the output file
with open(output_file, "w") as file:
    for edge in sorted(edge_set):  # Sort edges for consistency
        file.write(f"{edge[0]}\t{edge[1]}\n")

print(f"Edge list extracted and saved to {output_file}")

In [ ]:

# Step 2: Load the Deezer graph
# Assuming the edge list is in CSV format with two columns: source and target
G = nx.read_edgelist("Data/ca-CondMat.txt/edge_list.txt", nodetype=int)
LCC_nodes = max(nx.connected_components(G), key=len)
G = G.subgraph(LCC_nodes)
laplacian_sparse = nx.normalized_laplacian_matrix(G).astype(np.float64)

degrees = dict(G.degree())
# Compute the square root of the degrees
inv_sqrt_degrees = np.array([1 / np.sqrt(degrees[node]) if degrees[node] != 0 else 0 for node in G.nodes()])

# Step 4: Compute the smallest 6 eigenvalues of the Laplacian (for example)
num_eigenvalues = 10
eigenvalues, eigenvectors = eigsh(laplacian_sparse, k=num_eigenvalues, which="SM")


In [ ]:
K = 3
# Perform K-means clustering on the eigenvectors
kmeans = KMeans(n_clusters=K, n_init=10, random_state=42)
scaled_eigenvectors = inv_sqrt_degrees[:, np.newaxis] * eigenvectors

clusters = kmeans.fit_predict(scaled_eigenvectors[:, :K])

degrees = dict(G.degree())
# Compute the square root of the degrees
sqrt_degrees = np.array([np.sqrt(degrees[node]) for node in G.nodes()])

# Compute Rayleigh quotients for each cluster
rayleigh_quotients = []
indicator_vectors = np.zeros(eigenvectors.shape)

for cluster_idx in range(K):
    # Create indicator vector for the cluster
    indicator_vector = (clusters == cluster_idx).astype(float) * sqrt_degrees
    indicator_vector = indicator_vector / np.linalg.norm(indicator_vector, 2)
    indicator_vectors[:, cluster_idx] = indicator_vector
    # Compute Rayleigh quotient
    numerator = indicator_vector.T @ laplacian_sparse @ indicator_vector
    denominator = indicator_vector.T @ indicator_vector
    rayleigh_quotients.append(numerator / denominator)

rayleigh_quotients = sorted(rayleigh_quotients)
beta_K_by_K = indicator_vectors.T @ scaled_eigenvectors[:, 0:K]
combined_indicator_vectors = indicator_vectors @ beta_K_by_K
combined_indicator_vectors = gram_schmidt(combined_indicator_vectors)
combined_rayleigh_quotients = []
for i in range(K):
    numerator = combined_indicator_vectors[:, i].T @ laplacian_sparse @ combined_indicator_vectors[:, i]
    denominator = combined_indicator_vectors[:, i].T @ combined_indicator_vectors[:, i]
    combined_rayleigh_quotients.append(numerator / denominator)

combined_rayleigh_quotients = sorted(combined_rayleigh_quotients)

In [ ]:
collaborations_dataset_results = {}
true_val_matrix_combined = eigenvectors[:, :K] - indicator_vectors @ (
        indicator_vectors.T @ eigenvectors[:, :K])
true_val_combined = (1 / K) * np.linalg.norm(true_val_matrix_combined)

rayleigh_quotients = np.round(rayleigh_quotients, 6)
normalized_L_eigenvalues = np.round(eigenvalues, 6)

#b, v = apply_recursive_st_search(rayleigh_quotients[0:4], normalized_L_eigenvalues[0:4], 0, error = 0, bound_indices = [], values = [])
split, v = apply_recursive_st_brute_force(rayleigh_quotients, eigenvalues[0:K+1], K)


collaborations_dataset_results = {}
print("K: ", K)
collaborations_dataset_results["K"] = K
print("Number of vertices N: ", len(eigenvectors))
collaborations_dataset_results["N"] = len(eigenvectors)
print("Number of edges M: ", np.sum(list(degrees.values())) / 2)
collaborations_dataset_results["M"] = np.sum(list(degrees.values())) / 2
print("True value: ", true_val_combined)
collaborations_dataset_results["True Value"] = true_val_combined
print("M&S ST: ", max(np.diag(indicator_vectors.T @ laplacian_sparse @ indicator_vectors))/eigenvalues[K])
collaborations_dataset_results["M&S ST"] = max(np.diag(indicator_vectors.T @ laplacian_sparse @ indicator_vectors))/eigenvalues[K]
print("General ST: ", np.sum(rayleigh_quotients)/(K * eigenvalues[K]))
collaborations_dataset_results["General ST"] = np.sum(rayleigh_quotients)/(K * eigenvalues[K])
print("Recursive ST: ", v/K)
collaborations_dataset_results["Recursive ST"] = v/K

dataset_results["Collaborations (CA-CondMat)"] = collaborations_dataset_results

## Results

In [ ]:
pd.DataFrame(dataset_results).T

<h1 style="font-size: 50px">Directed Graphs</h1>

# More Imports

In [ ]:
import itertools
import urllib.request
from helpers import compute_volume, compute_weight_between_sets, Psi, compute_theta, compute_new_bound, compute_ls_bounds

# DSBMs
First we consider a DSBM with a 4 cycle for its cluster structure where we add noise to the clusters.

In [ ]:
def get_hermitian_adjacency_matrix(N, edges, root_of_unity):
    A = np.zeros((N, N), dtype = np.complex128)
    for edge in edges:
        u, v = edge
        A[u, v] = np.exp(2* 1j * np.pi / root_of_unity)
        A[v, u] = np.conj(A[u, v])
    return A

def get_adjacency_matrix(N, edges):
    A = np.zeros((N, N))
    for edge in edges:
        u, v = edge
        A[u, v] = 1
    return A

In [ ]:
# construct directed graph with 10 cycle
n = [0, 100, 200, 300, 400]
N = n[-1]
k = 4

bounds = {}

partition = []
for i in range(k):
    partition.append([j for j in range(n[i], n[i + 1])])
### FOR CYCLE
C = [(i, (i + 1) % k) for i in range(k)]
C_path = [(i, i + 1) for i in range(k - 1)]

F = np.array([[0.5, 1.0, 0.5, 0.0],
              [0.0, 0.5, 1.0, 0.5],
              [0.5, 0.0, 0.5, 1.0],
              [1.0, 0.5, 0.0, 0.5]])

def generate_P_cycle(eps=0.01):
    P = [[eps, 1.0, eps, 1.0],
         [1.0, eps, 1.0, eps],
         [eps, 1.0, eps, 1.0],
         [1.0, eps, 1.0, eps]]
    return np.array(P)

sample_size = 10 # SET TO 1 FOR TESTING

true_vals = []
new_bounds = []
new_bound_rayleigh_quotients = []
ls_bound_1s = []
ls_bound_2s = []
psis = []
lambda_1s = []
lambda_2s = []
rayleigh_quotients = []

for eps in np.arange(0, 0.2, 0.01):
    P = generate_P_cycle(eps)
    for _ in range(sample_size):
        edges = []
    
        for i in range(k):
            for j in range(k):
                if i == j:
                    prob_existing_edge = P[i, j]
                    for index, u in enumerate(partition[i]):
                        for v in partition[i][index + 1:]:
                            if u == v:
                                continue
                            if np.random.rand() <= prob_existing_edge:
                                if np.random.rand() <= F[i, j]:
                                    edges.append((u, v))
                                else:
                                    edges.append((v, u))
                else:
                    prob_existing_edge = P[i, j]
                    for u in partition[i]:
                        for v in partition[j]:
                            if np.random.rand() <= prob_existing_edge:
                                if np.random.rand() <= F[i, j]:
                                    edges.append((u, v))
                                else:
                                    edges.append((v, u))
    
    
        A = get_hermitian_adjacency_matrix(N = N, edges = edges, root_of_unity=k)
        degrees = np.sum(np.abs(A), axis=1)
        D = np.diag(degrees)
        D_sqrt = np.diag(np.sqrt(degrees))
        D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
        norm_L = np.eye(int(N)) - D_inv_sqrt @ A @ D_inv_sqrt
        
        # construct indicator vector
        ind_vector = np.zeros((N,), dtype = np.complex128)
        for i in range(k):
            ind_vector[n[i]:n[i+1]] = np.exp(-1j * 2 * np.pi * i / k)
        ind_vector = D_sqrt @ ind_vector
        ind_vector = ind_vector / np.linalg.norm(ind_vector)
        
        # compute rayleigh quotient of indicator vector
        chi_rayleigh_quotient = np.real((np.conj(ind_vector).T @ norm_L @ ind_vector) / (np.conj(ind_vector).T @ ind_vector))
        
    
        #Now computing using Laenen & Sun's choice of roots of unity
        k_ls = int(np.ceil(2 * np.pi * k))
        A_ls = get_hermitian_adjacency_matrix(N = N, edges = edges, root_of_unity=k_ls)
        D_ls = np.sum(np.abs(A_ls), axis=1)
        D_inv_sqrt_ls = np.diag(1 / np.sqrt(D_ls))
        norm_L_ls = np.eye(int(N)) - D_inv_sqrt_ls @ A_ls @ D_inv_sqrt_ls
    
        eigvals, eigvecs = np.linalg.eigh(norm_L)
        idx = eigvals.argsort()
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        ls_eigvals, ls_eigvecs = np.linalg.eigh(norm_L_ls)
        # sort
        idx = ls_eigvals.argsort()
        ls_eigvals = ls_eigvals[idx]
        ls_eigvecs = ls_eigvecs[:, idx]
        A_standard = get_adjacency_matrix(N, edges)
        ups = Psi(partition, A_standard, degrees, C)
        theta = compute_theta(partition, A_standard, degrees, C_path)
        
        # computing true value
        true_val = np.linalg.norm(eigvecs[:,0] - (np.conj(ind_vector).T @ eigvecs[:,0]) * ind_vector)**2
        
        new_bound_rayleigh_quotient = min((chi_rayleigh_quotient - eigvals[0])/ (eigvals[1] - eigvals[0]), chi_rayleigh_quotient / eigvals[1])
        new_bound = compute_new_bound(eigvals, ups)
        ls_bound_1, ls_bound_2 = compute_ls_bounds(ls_eigvals, theta, k)
        
        true_vals.append(true_val)
        new_bounds.append(new_bound)
        new_bound_rayleigh_quotients.append(new_bound_rayleigh_quotient)
        ls_bound_1s.append(ls_bound_1)
        ls_bound_2s.append(ls_bound_2)
        psis.append(ups)
        lambda_1s.append(eigvals[0])
        lambda_2s.append(eigvals[1])
        rayleigh_quotients.append(chi_rayleigh_quotient)

    true_val = np.mean(true_vals)
    new_bound = np.mean(new_bounds)
    new_bound_rayleigh_quotient = np.mean(new_bound_rayleigh_quotients)
    ls_bound_1 = np.mean(ls_bound_1s)
    ls_bound_2 = np.mean(ls_bound_2s)
    ups = np.mean(psis)
    lambda_1 = np.mean(lambda_1s)
    lambda_2 = np.mean(lambda_2s)
    chi_rayleigh_quotient = np.mean(rayleigh_quotients)
    
    bounds[eps] = {'true_val': true_val,
                   'new_bound': new_bound,
                   'new_bound_rayleigh_quotient':new_bound_rayleigh_quotient,
                   'ls_bound_1': ls_bound_1,
                   'ls_bound_2': ls_bound_2,
                   'Psi': ups,
                   'lambda_1': eigvals[0],
                   'lambda_2': eigvals[1],
                   'rayleigh quotient': chi_rayleigh_quotient}
bounds_df = pd.DataFrame(bounds).T
bounds_df.columns = [r'$\|f_1 - \alpha \chi\|^2$',
                     r'$\frac{4 \Psi - \lambda_1}{\lambda_2 - \lambda_1}$',
                      r'$\frac{\chi^* \mathcal{L} \chi - \lambda_1}{\lambda_2 - \lambda_1}$',
                     r'$\frac{1}{\eta_k}$',
                     r'$\frac{1}{\eta_4-1}$',
                     'ups',
                     'lambda_1',
                     'lambda_2',
                     'rayleigh quotient']

In [ ]:
df_for_plot = bounds_df[[r'$\|f_1 - \alpha \chi\|^2$', r'$\frac{4 \Psi - \lambda_1}{\lambda_2 - \lambda_1}$', r'$\frac{\chi^* \mathcal{L} \chi - \lambda_1}{\lambda_2 - \lambda_1}$', r'$\frac{1}{\eta_4-1}$']][
    (bounds_df[r'$\frac{4 \Psi - \lambda_1}{\lambda_2 - \lambda_1}$'] <= 0.8) + (bounds_df[r'$\frac{1}{\eta_4-1}$'] <= 0.8)].iloc[1:,:]
df_for_plot.columns = ['Actual Error'+ r'\n$\|f_1 - \alpha \chi\|^2$',  
                       'Our bound ' + r'\n$\frac{4 \Psi - \lambda_1}{\lambda_2 - \lambda_1}$', 
                       'Our bound (Rayleigh Quotient) '+ r'\n$\frac{\chi^* \mathcal{L} \chi - \lambda_1}{\lambda_2 - \lambda_1}$',
                       'LS bound ' + r'\n$\frac{1}{\eta_4-1}$']

# Plot the dataframe
ax = df_for_plot.plot(
     marker='o', linestyle='-', color=['purple','blue','green', 'red'],
    figsize=(8
             ,5), logy=True) #title=r'Cyclic DSBM (k=4)',

# Set labels and grid
plt.xlabel(r'Noise $\epsilon$', fontsize=20)
plt.ylabel(r'Error', fontsize=20)
plt.grid(True)

legend_labels = [
    'Actual',  
    'Ours' + r' ($\Psi$)', 
    'Ours (Rayleigh)',
    'LS bound'
    ]
# Set custom legend with smaller font for equations
plt.legend(
    legend_labels, fontsize=20, loc='upper left', bbox_to_anchor=(1, 1),labelspacing=1.5, ncol=1)

# Save the figure
plt.savefig('Figures/4_cluster_cycle_noise.png', bbox_inches='tight')
plt.show()

Now we consider a similar graph but where the cluster structure is a path of 4 clusters rather than a cycle.

In [ ]:
# construct directed graph with 10 cycle
n = [0, 100, 200, 300, 400]
N = n[-1]
k = 4

bounds = {}

partition = []
for i in range(k):
    partition.append([j for j in range(n[i], n[i + 1])])
### FOR CYCLE
C = [(i, (i + 1) % k) for i in range(k)]
C_path = [(i, i + 1) for i in range(k - 1)]

F = np.array([[0.5, 1.0, 0.5, 0.5],
              [0.0, 0.5, 1.0, 0.5],
              [0.5, 0.0, 0.5, 1.0],
              [0.5, 0.5, 0.0, 0.5]])

def generate_P_cycle(eps=0.01):
    P = [[eps, 1.0, eps, eps],
         [1.0, eps, 1.0, eps],
         [eps, 1.0, eps, 1.0],
         [eps, eps, 1.0, eps]]
    return np.array(P)

sample_size = 10 # SET TO 1 FOR TESTING

true_vals = []
new_bounds = []
new_bound_rayleigh_quotients = []
ls_bound_1s = []
ls_bound_2s = []
psis = []
lambda_1s = []
lambda_2s = []
rayleigh_quotients = []

for eps in np.arange(0, 0.2, 0.01):
    P = generate_P_cycle(eps)
    for _ in range(sample_size):
        edges = []
    
        for i in range(k):
            for j in range(k):
                if i == j:
                    prob_existing_edge = P[i, j]
                    for index, u in enumerate(partition[i]):
                        for v in partition[i][index + 1:]:
                            if u == v:
                                continue
                            if np.random.rand() <= prob_existing_edge:
                                if np.random.rand() <= F[i, j]:
                                    edges.append((u, v))
                                else:
                                    edges.append((v, u))
                else:
                    prob_existing_edge = P[i, j]
                    for u in partition[i]:
                        for v in partition[j]:
                            if np.random.rand() <= prob_existing_edge:
                                if np.random.rand() <= F[i, j]:
                                    edges.append((u, v))
                                else:
                                    edges.append((v, u))
    
    
        A = get_hermitian_adjacency_matrix(N = N, edges = edges, root_of_unity=k)
        degrees = np.sum(np.abs(A), axis=1)
        D = np.diag(degrees)
        D_sqrt = np.diag(np.sqrt(degrees))
        D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
        norm_L = np.eye(int(N)) - D_inv_sqrt @ A @ D_inv_sqrt
        
        # construct indicator vector
        ind_vector = np.zeros((N,), dtype = np.complex128)
        for i in range(k):
            ind_vector[n[i]:n[i+1]] = np.exp(-1j * 2 * np.pi * i / k)
        ind_vector = D_sqrt @ ind_vector
        ind_vector = ind_vector / np.linalg.norm(ind_vector)
        
        # compute rayleigh quotient of indicator vector
        chi_rayleigh_quotient = np.real((np.conj(ind_vector).T @ norm_L @ ind_vector) / (np.conj(ind_vector).T @ ind_vector))
    

        #Now computing using Laenen & Sun's choice of roots of unity
        k_ls = int(np.ceil(2 * np.pi * k))
        A_ls = get_hermitian_adjacency_matrix(N = N, edges = edges, root_of_unity=k_ls)
        D_ls = np.sum(np.abs(A_ls), axis=1)
        D_inv_sqrt_ls = np.diag(1 / np.sqrt(D_ls))
        norm_L_ls = np.eye(int(N)) - D_inv_sqrt_ls @ A_ls @ D_inv_sqrt_ls
    
        eigvals, eigvecs = np.linalg.eigh(norm_L)
        idx = eigvals.argsort()
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        ls_eigvals, ls_eigvecs = np.linalg.eigh(norm_L_ls)
        # sort
        idx = ls_eigvals.argsort()
        ls_eigvals = ls_eigvals[idx]
        ls_eigvecs = ls_eigvecs[:, idx]
        A_standard = get_adjacency_matrix(N, edges)
        ups = Psi(partition, A_standard, degrees, C)
        theta = compute_theta(partition, A_standard, degrees, C_path)
        
        # computing true value
        true_val = np.linalg.norm(eigvecs[:,0] - (np.conj(ind_vector).T @ eigvecs[:,0]) * ind_vector)**2
    
        new_bound_rayleigh_quotient = min((chi_rayleigh_quotient - eigvals[0])/ (eigvals[1] - eigvals[0]), chi_rayleigh_quotient / eigvals[1])
        new_bound = compute_new_bound(eigvals, ups)
        ls_bound_1, ls_bound_2 = compute_ls_bounds(ls_eigvals, theta, k)
        
        true_vals.append(true_val)
        new_bounds.append(new_bound)
        new_bound_rayleigh_quotients.append(new_bound_rayleigh_quotient)
        ls_bound_1s.append(ls_bound_1)
        ls_bound_2s.append(ls_bound_2)
        psis.append(ups)
        lambda_1s.append(eigvals[0])
        lambda_2s.append(eigvals[1])
        rayleigh_quotients.append(chi_rayleigh_quotient)

    true_val = np.mean(true_vals)
    new_bound = np.mean(new_bounds)
    new_bound_rayleigh_quotient = np.mean(new_bound_rayleigh_quotients)
    ls_bound_1 = np.mean(ls_bound_1s)
    ls_bound_2 = np.mean(ls_bound_2s)
    ups = np.mean(psis)
    lambda_1 = np.mean(lambda_1s)
    lambda_2 = np.mean(lambda_2s)
    chi_rayleigh_quotient = np.mean(rayleigh_quotients)
    
    bounds[eps] = {'true_val': true_val,
                   'new_bound': new_bound,
                   'new_bound_rayleigh_quotient':new_bound_rayleigh_quotient,
                   'ls_bound_1': ls_bound_1,
                   'ls_bound_2': ls_bound_2,
                   'Psi': ups,
                   'lambda_1': eigvals[0],
                   'lambda_2': eigvals[1],
                   'rayleigh quotient': chi_rayleigh_quotient}
bounds_df = pd.DataFrame(bounds).T
bounds_df.columns = [r'$\|f_1 - \alpha \chi\|^2$',
                     r'$\frac{4 \Psi - \lambda_1}{\lambda_2 - \lambda_1}$',
                      r'$\frac{\chi^* \mathcal{L} \chi - \lambda_1}{\lambda_2 - \lambda_1}$',
                     r'$\frac{1}{\eta_k}$',
                     r'$\frac{1}{\eta_4-1}$',
                     'ups',
                     'lambda_1',
                     'lambda_2',
                     'rayleigh quotient']


In [ ]:
df_for_plot = bounds_df[[r'$\|f_1 - \alpha \chi\|^2$', r'$\frac{4 \Psi - \lambda_1}{\lambda_2 - \lambda_1}$', r'$\frac{\chi^* \mathcal{L} \chi - \lambda_1}{\lambda_2 - \lambda_1}$', r'$\frac{1}{\eta_4-1}$']][
    (bounds_df[r'$\frac{4 \Psi - \lambda_1}{\lambda_2 - \lambda_1}$'] <= 0.8) + (bounds_df[r'$\frac{1}{\eta_4-1}$'] <= 0.8)].iloc[1:,:]
df_for_plot.columns = ['Actual Error'+ r'\n$\|f_1 - \alpha \chi\|^2$',  
                       'Our bound ' + r'\n$\frac{4 \Psi - \lambda_1}{\lambda_2 - \lambda_1}$', 
                       'Our bound (Rayleigh Quotient) '+ r'\n$\frac{\chi^* \mathcal{L} \chi - \lambda_1}{\lambda_2 - \lambda_1}$',
                       'LS bound ' + r'\n$\frac{1}{\eta_4-1}$']

# Plot the dataframe
ax = df_for_plot.plot(
     marker='o', linestyle='-', color=['purple','blue','green', 'red'],
    figsize=(8
             ,5), logy=True) #title=r'Path DSBM (k=4)',

# Set labels and grid
plt.xlabel(r'Noise $\epsilon$', fontsize=20)
plt.ylabel(r'Error', fontsize=20)
plt.grid(True)

legend_labels = [
    'Actual',  
    'Ours' + r' ($\Psi$)', 
    'Ours (Rayleigh)',
    'LS bound'
    ]

# Set custom legend with smaller font for equations
plt.legend(
    legend_labels, fontsize=20, loc='upper left', bbox_to_anchor=(1, 1),labelspacing=1.5, ncol=1)

# Save the figure
plt.savefig('Figures/4_cluster_path_noise.png',bbox_inches='tight')
plt.show()

# YellowStone Graph
In this section we produce the plot the bounds and some figures for our yellowstone graph. Since k is small for this graph (and the others), we will search for the best permutation of the clusters by brute force for computing $\Psi$ and $\eta$.

In [ ]:
# Nodes and edges for the second image
nodes = [
    "Plants,\n Flowers,\n Nuts,\n Seeds,\n Fruit,\n Insects", "Pika", "Red-breasted\n nuthatch",
    "Pacific\n tree frog", "Edith's\n checkerspot", "Douglas'\n squirrel", "Mule deer",
    "Black-tipped\n jackrabbit", "Pine marten", "Western\n whiptail", "Raven",
    "Ringtail", "Coyote", "Mountain\n lion", "Bobcat"
]

# Initialize the adjacency matrix with zeros
adj_matrix = np.zeros((len(nodes), len(nodes)), dtype=int)

# Map the nodes to matrix index
index_map = {species: i for i, species in enumerate(nodes)}

partition = {"Producers and Decomposer": ["Plants,\n Flowers,\n Nuts,\n Seeds,\n Fruit,\n Insects"],
             "Primary Consumers": ["Pika", "Red-breasted\n nuthatch", "Pacific\n tree frog", "Edith's\n checkerspot",
                                   "Douglas'\n squirrel", "Mule deer"],
             "Secondary Consumers": ["Black-tipped\n jackrabbit", "Pine marten", "Western\n whiptail", "Raven", "Ringtail"],
             "Tertiary Consumers": ["Coyote", "Mountain\n lion", "Bobcat"]}

partition_numbered = [[index_map[species] for species in community] for community in partition.values()]
# Manually add edges based on the image (directed edges where energy is transferred)
edges = [
    ("Plants,\n Flowers,\n Nuts,\n Seeds,\n Fruit,\n Insects", "Pika"),
    ("Plants,\n Flowers,\n Nuts,\n Seeds,\n Fruit,\n Insects", "Red-breasted\n nuthatch"),
    ("Plants,\n Flowers,\n Nuts,\n Seeds,\n Fruit,\n Insects", "Pacific\n tree frog"),
    ("Plants,\n Flowers,\n Nuts,\n Seeds,\n Fruit,\n Insects", "Edith's\n checkerspot"),
    ("Plants,\n Flowers,\n Nuts,\n Seeds,\n Fruit,\n Insects", "Douglas'\n squirrel"),
    ("Plants,\n Flowers,\n Nuts,\n Seeds,\n Fruit,\n Insects", "Mule deer"),

    ("Pika", "Ringtail"),
    ("Pika", "Western\n whiptail"),
    ("Pika", "Raven"),
    ("Pika", "Black-tipped\n jackrabbit"),
    ("Pika", "Pine marten"),

    ("Red-breasted\n nuthatch", "Western\n whiptail"),

    ("Pacific\n tree frog", "Ringtail"),
    ("Pacific\n tree frog", "Western\n whiptail"),
    ("Pacific\n tree frog", "Raven"),
    ("Pacific\n tree frog", "Black-tipped\n jackrabbit"),
    ("Pacific\n tree frog", "Pine marten"),

    ("Edith's\n checkerspot", "Western\n whiptail"),
    ("Edith's\n checkerspot", "Raven"),
    ("Edith's\n checkerspot", "Black-tipped\n jackrabbit"),

    ("Douglas'\n squirrel", "Ringtail"),
    ("Douglas'\n squirrel", "Raven"),
    ("Douglas'\n squirrel", "Black-tipped\n jackrabbit"),
    ("Douglas'\n squirrel", "Pine marten"),

    ("Mule deer", "Mountain\n lion"),
    ("Mule deer", "Coyote"),

    ("Ringtail", "Coyote"),
    ("Ringtail", "Mountain\n lion"),
    ("Ringtail", "Bobcat"),

    ("Western\n whiptail", "Mountain\n lion"),
    ("Western\n whiptail", "Bobcat"),
    ("Western\n whiptail", "Coyote"),

    ("Black-tipped\n jackrabbit", "Mountain\n lion"),
    ("Black-tipped\n jackrabbit", "Coyote"),
    ("Black-tipped\n jackrabbit", "Bobcat"),

    ("Pine marten", "Mountain\n lion"),
    ("Pine marten", "Bobcat")
]

# Now we can fill the adjacency matrix again based on these edges
for source, target in edges:
    adj_matrix[index_map[source], index_map[target]] = 1

# Create a DataFrame for better visualization
adj_matrix_df = pd.DataFrame(adj_matrix, index=nodes, columns=nodes)
G = nx.DiGraph()

# Add edges to the graph from the previous trophic relationships
G.add_edges_from(edges)

pos = {
    "Plants,\n Flowers,\n Nuts,\n Seeds,\n Fruit,\n Insects": (0, 6),

    "Pika": (-6, 4), "Red-breasted\n nuthatch": (-3, 4), "Pacific\n tree frog": (0, 4), 
    "Edith's\n checkerspot": (3, 4), "Douglas'\n squirrel": (6, 4), "Mule deer": (9, 4),

    "Black-tipped\n jackrabbit": (-7, 2), "Pine marten": (-4, 2), 
    "Western\n whiptail": (-1, 2), "Raven": (2, 2), "Ringtail": (5, 2),

    "Coyote": (-5, 0), "Mountain\n lion": (-2, 0), "Bobcat": (1, 0)
}


# Color mapping for different communities
color_map = {
    "Producers and Decomposer": "green",
    "Primary Consumers": "blue",
    "Secondary Consumers": "orange",
    "Tertiary Consumers": "red"
}

# Assign colors based on community
node_colors = []
for node in G.nodes():
    for community, members in partition.items():
        if node in members:
            node_colors.append(color_map[community])

# Plot the graph using networkx
plt.figure(figsize=(10, 8))
nx.draw_networkx(G, pos, with_labels=True, node_size=5000, 
                 font_size=10, font_color='black', font_weight='bold',node_color='white', edgecolors=node_colors, arrows=True, alpha=0.65)

# Display the plot
#plt.title('Directed Graph of Yellowstone Trophic Cascade')
plt.axis('off')

plt.savefig('Figures/YellowstoneTrophicCascade2.png', bbox_inches='tight')
plt.show()



In [ ]:
# compute hermitian adjacency matrix
k = 4
k_ls = int(np.ceil(2 * np.pi * k))
A = adj_matrix * np.exp(1j * 2 * np.pi / 4) + adj_matrix.T * np.exp(-1j * 2 * np.pi / 4)
A_ls = adj_matrix * np.exp(1j * 2 * np.pi / k_ls) + adj_matrix.T * np.exp(-1j * 2 * np.pi / k_ls)
# compute degree matrix
degrees = np.sum(np.abs(A), axis=1)
D = np.diag(degrees)
D_sqrt = np.diag(np.sqrt(degrees))
D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
norm_L = np.eye(len(nodes)) - D_inv_sqrt @ A @ D_inv_sqrt

#Compute indicator vector
ind_vector = np.zeros((len(nodes),), dtype = np.complex128)
for i in range(k):
    for node in partition[list(partition.keys())[i]]:
        ind_vector[index_map[node]] = np.exp(-1j * 2 * np.pi * i / k)
ind_vector = D_sqrt @ ind_vector
ind_vector = ind_vector / np.linalg.norm(ind_vector)

rayleigh_quotient = np.real((np.conj(ind_vector).T @ norm_L @ ind_vector) / (np.conj(ind_vector).T @ ind_vector))

#Now computing using Laenen & Sun's choice of roots of unity
D_ls = np.sum(np.abs(A_ls), axis=1)
D_inv_sqrt_ls = np.diag(1 / np.sqrt(D_ls))
norm_L_ls = np.eye(len(nodes)) - D_inv_sqrt_ls @ A_ls @ D_inv_sqrt_ls
evals, evecs = np.linalg.eigh(norm_L)
evals_ls, evecs_ls = np.linalg.eigh(norm_L_ls)
# plot first evec

thetas = []
for perm in itertools.permutations(range(4)):
    C = [(perm[i], perm[i + 1]) for i in range(3)]
    theta = compute_theta(partition_numbered, adj_matrix, degrees, C)
    thetas.append(theta)

theta = max(thetas)
max_perm_theta = list(itertools.permutations(range(4)))[np.argmax(thetas)]

C = [(0, 1), (1, 2), (2, 3), (3, 0)]

In [ ]:
plt.figure(figsize=(10, 10))

plt.plot(evecs[:, 0].real, evecs[:, 0].imag, '+', label='standard', alpha=0.5)
# add labels
for i, txt in enumerate(nodes):
    plt.annotate(txt, (evecs[i, 0].real, evecs[i, 0].imag))

# colour by community
for i, community in enumerate(partition_numbered):
    plt.plot(evecs[community, 0].real, evecs[community, 0].imag, 'o', label=f'community {i}', alpha=0.5)

In [ ]:
true_val = np.linalg.norm(evecs[:, 0] - (np.conj(ind_vector).T @ evecs[:, 0]) * ind_vector) ** 2
theta = compute_theta(partition_numbered, adj_matrix, degrees, [(i, i + 1) for i in range(4)])
ls_1, ls_2 = compute_ls_bounds(evals_ls, theta, k)
print("Theta is ", theta)
print("1/eta_k is ", ls_1)
print("1/(eta_k-1) is ", ls_2)
psi = Psi(partition_numbered, adj_matrix, degrees, C)
new_bound = compute_new_bound(evals, psi)
new_bound_with_rayleigh_quotient = (rayleigh_quotient - evals[0]) / (evals[1] - evals[0])
print("Rayleigh Quotient is ", rayleigh_quotient)
print("Psi is ", psi)
print("True value is ", true_val)
print("New bound is ", new_bound)
print("New bound with rayleigh quotient is ", new_bound_with_rayleigh_quotient)

# Covid Infection Graph
In this section we compute the covid infection graph. Since the graph fits a perfect 4-cycle, we can state exactly what the cycle C should be. 

In [ ]:
patient_info_df = pd.read_csv("Data/PatientInfo.csv")
patient_info_df.head()
edges_df = patient_info_df[["infected_by", "patient_id"]].dropna(subset=["infected_by"])
edges_df.columns = ["source", "target"]
edges_df = edges_df.astype(str)
# create adjacency matrix
index = list(set(edges_df["source"].values) | set(edges_df["target"].values))
index.sort()
adjacency_matrix = np.zeros((len(index), len(index)))
for i, row in edges_df.iterrows():
    source = row["source"]
    target = row["target"]
    source_index = index.index(source)
    target_index = index.index(target)
    adjacency_matrix[source_index, target_index] = 1
adjacency_matrix_df = pd.DataFrame(adjacency_matrix, index=index, columns=index)


In [ ]:

def largest_connected_component_adjacency(df):
    # Convert the DataFrame to a NumPy array
    adj_matrix = df

    # Create a directed graph from the adjacency matrix
    G = nx.from_pandas_adjacency(adj_matrix, create_using=nx.DiGraph)

    # Find the largest strongly connected component
    largest_wcc = max(nx.weakly_connected_components(G), key=len)

    # Create a subgraph from the largest SCC
    largest_wcc_subgraph = G.subgraph(largest_wcc)

    # Get the adjacency matrix of the largest SCC subgraph
    largest_wcc_adj_matrix = nx.to_numpy_array(largest_wcc_subgraph, nodelist=sorted(largest_wcc))

    # Create a new DataFrame for the adjacency matrix of the largest SCC
    largest_wcc_df = pd.DataFrame(largest_wcc_adj_matrix,
                                  index=sorted(largest_wcc),
                                  columns=sorted(largest_wcc))

    return largest_wcc_df

In [ ]:
largest_wcc_df = largest_connected_component_adjacency(adjacency_matrix_df)
# draw graph
G = nx.from_pandas_adjacency(largest_wcc_df, create_using=nx.DiGraph)

k = 4
herm_A = np.exp(1j * 2 * np.pi / k) * largest_wcc_df.values + np.exp(-1j * 2 * np.pi / k) * largest_wcc_df.values.T
degrees = np.sum(np.abs(herm_A), axis=1)
D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
norm_L = np.eye(len(herm_A)) - D_inv_sqrt @ herm_A @ D_inv_sqrt

k_ls = int(np.ceil(2 * np.pi * k))
herm_A_ls = np.exp(1j * 2 * np.pi / k_ls) * largest_wcc_df.values + np.exp(
    -1j * 2 * np.pi / k_ls) * largest_wcc_df.values.T
degrees_ls = np.sum(np.abs(herm_A_ls), axis=1)
D_inv_sqrt_ls = np.diag(1 / np.sqrt(degrees_ls))
norm_L_ls = np.eye(len(herm_A_ls)) - D_inv_sqrt_ls @ herm_A_ls @ D_inv_sqrt_ls

# Compute Eigenvectors and Eigenvalues
e_vals, e_vecs = np.linalg.eigh(norm_L)
e_vals_ls, e_vecs_ls = np.linalg.eigh(norm_L_ls)


datapoints = np.column_stack((D_inv_sqrt @ e_vecs[:, 0].real, D_inv_sqrt @ e_vecs[:, 0].imag))
kmeans = KMeans(n_clusters=4, random_state=0).fit(datapoints)
partition_numbered = [np.where(kmeans.labels_ == i)[0] for i in range(4)]

color_map = []
for i, node in enumerate(largest_wcc_df.index):
    if i in partition_numbered[0]:
        color_map.append('blue')
    elif i in partition_numbered[1]:
        color_map.append('red')
    elif i in partition_numbered[2]:
        color_map.append('green')
    elif i in partition_numbered[3]:
        color_map.append('yellow')
    else:
        color_map.append('black')
G = nx.from_pandas_adjacency(largest_wcc_df, create_using=nx.DiGraph)
pos = {}
partition_sizes = [len(partition) for partition in partition_numbered]
scale = 60
for i, node_number in enumerate(largest_wcc_df.index):
    if i in partition_numbered[0]:
        index_in_partition = np.where(partition_numbered[0] == i)[0][0]
        pos[node_number] = (10 + scale * index_in_partition / partition_sizes[0], 0)
    elif i in partition_numbered[1]:
        index_in_partition = np.where(partition_numbered[1] == i)[0][0]
        pos[node_number] = (0, 10 + scale * index_in_partition / partition_sizes[0])
    elif i in partition_numbered[2]:
        index_in_partition = np.where(partition_numbered[2] == i)[0][0]
        pos[node_number] = (0, -10 + scale * index_in_partition / partition_sizes[0])
    elif i in partition_numbered[3]:
        index_in_partition = np.where(partition_numbered[3] == i)[0][0]
        pos[node_number] = (-10 + scale * index_in_partition / partition_sizes[0], 0)
    else:
        pos[node_number] = (0, 0)

plt.figure(figsize=(8, 8))
nx.draw(G, pos, node_color=color_map, with_labels=False, node_size=15, width=0.5, connectionstyle='arc3, rad = 0.1')
#plt.title("Partitioning of Largest Connected Component of DS4C Infection Network")
plt.savefig('Figures/DS4CInfectionNetwork.png', bbox_inches='tight')
# compute weight between every pair of clusters

C = [(2, 0), (0, 1), (1, 3), (3, 2)]

theta = compute_theta(partition_numbered, largest_wcc_df.values, degrees, C)
compute_ls_bounds(e_vals_ls, theta, k)
eta_k = e_vals_ls[1] / (1 - (4 / k) * theta)

print("Number of vertices: ", len(largest_wcc_df))
print("Number of Edges: ", np.sum(np.sum(largest_wcc_df, axis=1)))

print("Theta", theta)
print("eta_k", e_vals_ls[1] / (1 - (4 / k) * theta))
print("1/eta_k", 1/eta_k)
print("1/(eta_k-1)", 1/(eta_k - 1))

print("Psi", Psi(partition_numbered, largest_wcc_df.values, degrees, C))
print("New bound", compute_new_bound(e_vals, Psi(partition_numbered, largest_wcc_df.values, degrees, C)))


# Some Other Real-World Directed Graphs
We now consider some real world directed graphs (ecological networks) from the cosinproject. We start with the Ythan Estuary dataset.

In [ ]:

# import edge list from http://cosinproject.eu/extra/data/foodwebs/ythan.txt

urllib.request.urlretrieve("http://cosinproject.eu/extra/data/foodwebs/ythan.txt", "Data/ythan.txt")
# construct adjacency matrix
with open("Data/ythan.txt", "r") as f:
    lines = f.readlines()
    edges = [tuple(map(int, line.strip().split())) for line in lines]

edges_df = pd.DataFrame(edges, columns=["source", "target"])
edges_df_unweighted = edges_df.applymap(int)
edges_df_unweighted.head(10)
# convert edges_df_unweighted to adjacency matrix
index = list(set(edges_df_unweighted["source"].values) | set(edges_df_unweighted["target"].values))
index.sort()
adj_matrix = np.zeros((len(index), len(index)))
for i, row in edges_df_unweighted.iterrows():
    source = row["source"]
    target = row["target"]
    source_index = index.index(source)
    target_index = index.index(target)
    adj_matrix[source_index, target_index] = 1

k = 4
k_ls = int(np.ceil(2 * np.pi * k))
herm_A = np.exp(1j * 2 * np.pi / k) * adj_matrix + np.exp(-1j * 2 * np.pi / k) * adj_matrix.T
herm_A_ls = np.exp(1j * 2 * np.pi / k_ls) * adj_matrix + np.exp(-1j * 2 * np.pi / k_ls) * adj_matrix.T
degrees = np.sum(np.abs(herm_A), axis=1)
D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
D_sqrt = np.diag(np.sqrt(degrees))
norm_L = np.eye(len(adj_matrix)) - D_inv_sqrt @ herm_A @ D_inv_sqrt
norm_L_ls = np.eye(len(adj_matrix)) - D_inv_sqrt @ herm_A_ls @ D_inv_sqrt

eigvals, eigvecs = np.linalg.eigh(norm_L)
idx = eigvals.argsort()
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

eigvals_ls, eigvecs_ls = np.linalg.eigh(norm_L_ls)
idx = eigvals_ls.argsort()
eigvals_ls = eigvals_ls[idx]
eigvecs_ls = eigvecs_ls[:, idx]

# cluster using first eigenvector
datapoints = np.column_stack((D_inv_sqrt @ eigvecs[:, 0].real, D_inv_sqrt @ eigvecs[:, 0].imag))
kmeans = KMeans(n_clusters=k, random_state=0).fit(datapoints)
partition_numbered = [np.where(kmeans.labels_ == i)[0] for i in range(k)]

rayleigh_quotients_all = []
for perm in itertools.permutations(range(k)):
    # construct indicator vector of clusters
    ind_vector = np.zeros((len(adj_matrix),), dtype=np.complex128)

    for i, cluster in enumerate(partition_numbered):
        ind_vector[cluster] = np.exp(2 * np.pi * 1j * perm[i] / k)

    ind_vector = D_sqrt @ ind_vector
    ind_vector = ind_vector / np.linalg.norm(ind_vector)

    # rotate indicator vector to align with first eigenvector
    alpha = np.conj(ind_vector).T @ eigvecs[:, 0]
    ind_vector = alpha * ind_vector
    ind_vector = ind_vector / np.linalg.norm(ind_vector)

    rayleigh_quotients_all.append(np.real(np.conj(ind_vector).T @ norm_L @ ind_vector))


# set indicator vector to be the one with the minimum rayleigh quotient
min_perm = list(itertools.permutations(range(k)))[rayleigh_quotients_all.index(min(rayleigh_quotients_all))]
min_ind_vector = np.zeros((len(adj_matrix),), dtype=np.complex128)

for i, cluster in enumerate(partition_numbered):
    min_ind_vector[cluster] = np.exp(2 * np.pi * 1j * min_perm[i] / k)

min_ind_vector = D_sqrt @ min_ind_vector
min_ind_vector = min_ind_vector / np.linalg.norm(min_ind_vector)

# compute Psi
Psis = []
for perm in itertools.permutations(range(k)):
    C = [(perm[i], perm[i + 1]) for i in range(k - 1)] + [(perm[k - 1], perm[0])]
    Psis.append(Psi(partition_numbered, adj_matrix, degrees, C))

min_perm_psi = list(itertools.permutations(range(k)))[Psis.index(min(Psis))]

# rotate indicator vector to align with first eigenvector
alpha = np.conj(min_ind_vector).T @ eigvecs[:, 0]
min_ind_vector = alpha * min_ind_vector

thetas = []

for perm in itertools.permutations(range(k)):
    C = [(perm[i], perm[i + 1]) for i in range(len(perm) - 1)]
    C.append((perm[-1], perm[0]))
    theta = compute_theta(partition_numbered, adj_matrix, degrees, C)
    thetas.append(theta)
    
max_perm_theta = list(itertools.permutations(range(k)))[thetas.index(max(thetas))]
print("Max perm theta ", max_perm_theta)
C = [(max_perm_theta[i], max_perm_theta[i + 1]) for i in range(len(max_perm_theta) - 1)]
theta = compute_theta(partition_numbered, adj_matrix, degrees, C)

compute_ls_bounds(eigvals_ls, theta, k)
eigvals_ls[1] / (1 - (4 / k) * theta)
eta = eigvals_ls[1] / (1 - (4 / k) * theta)

print("Number of vertices: ", len(adj_matrix))
print("Number of Edges: ", np.sum(np.sum(adj_matrix, axis=1)))
print("Theta", theta)
print("1/eta_k", 1 / eta)
print("1/(eta_k-1)", 1 / (eta - 1))
print("Psi", min(Psis))
print("rayleigh quotient", min(rayleigh_quotients_all))
print("Our bound (Rayleigh Quotient) ",(min(rayleigh_quotients_all) - eigvals[0]) / (eigvals[1] - eigvals[0]))
print("Our bound (Psi) ", (4*min(Psis) - eigvals[0])/ (eigvals[1] - eigvals[0]))
print("True value ", np.linalg.norm(min_ind_vector - eigvecs[:, 0]) ** 2)

## St.Martins Island
using k = 4.

In [ ]:

urllib.request.urlretrieve("http://cosinproject.eu/extra/data/foodwebs/stmartin.txt", "Data/stmartin.txt")
# construct adjacency matrix
with open("Data/stmartin.txt", "r") as f:
    lines = f.readlines()
    edges = [tuple(map(int, line.strip().split())) for line in lines]

edges_df = pd.DataFrame(edges, columns=["source", "target"])
edges_df_unweighted = edges_df.applymap(int)
edges_df_unweighted.head(10)
# convert edges_df_unweighted to adjacency matrix
index = list(set(edges_df_unweighted["source"].values) | set(edges_df_unweighted["target"].values))
index.sort()
adj_matrix = np.zeros((len(index), len(index)))
for i, row in edges_df_unweighted.iterrows():
    source = row["source"]
    target = row["target"]
    source_index = index.index(source)
    target_index = index.index(target)
    adj_matrix[source_index, target_index] = 1

k = 4
k_ls = int(np.ceil(2 * np.pi * k))
herm_A = np.exp(1j * 2 * np.pi / k) * adj_matrix + np.exp(-1j * 2 * np.pi / k) * adj_matrix.T
herm_A_ls = np.exp(1j * 2 * np.pi / k_ls) * adj_matrix + np.exp(-1j * 2 * np.pi / k_ls) * adj_matrix.T
degrees = np.sum(np.abs(herm_A), axis=1)
D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
D_sqrt = np.diag(np.sqrt(degrees))
norm_L = np.eye(len(adj_matrix)) - D_inv_sqrt @ herm_A @ D_inv_sqrt
norm_L_ls = np.eye(len(adj_matrix)) - D_inv_sqrt @ herm_A_ls @ D_inv_sqrt

eigvals, eigvecs = np.linalg.eigh(norm_L)
idx = eigvals.argsort()
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

eigvals_ls, eigvecs_ls = np.linalg.eigh(norm_L_ls)
idx = eigvals_ls.argsort()
eigvals_ls = eigvals_ls[idx]
eigvecs_ls = eigvecs_ls[:, idx]

# cluster using first eigenvector
datapoints = np.column_stack((D_inv_sqrt @ eigvecs[:, 0].real, D_inv_sqrt @ eigvecs[:, 0].imag))
kmeans = KMeans(n_clusters=k, random_state=0).fit(datapoints)
partition_numbered = [np.where(kmeans.labels_ == i)[0] for i in range(k)]

rayleigh_quotients_all = []
for perm in itertools.permutations(range(k)):
    # construct indicator vector of clusters
    ind_vector = np.zeros((len(adj_matrix),), dtype=np.complex128)

    for i, cluster in enumerate(partition_numbered):
        ind_vector[cluster] = np.exp(2 * np.pi * 1j * perm[i] / k)

    ind_vector = D_sqrt @ ind_vector
    ind_vector = ind_vector / np.linalg.norm(ind_vector)

    # rotate indicator vector to align with first eigenvector
    alpha = np.conj(ind_vector).T @ eigvecs[:, 0]
    ind_vector = alpha * ind_vector
    ind_vector = ind_vector / np.linalg.norm(ind_vector)

    rayleigh_quotients_all.append(np.real(np.conj(ind_vector).T @ norm_L @ ind_vector))


# set indicator vector to be the one with the minimum rayleigh quotient
min_perm = list(itertools.permutations(range(k)))[rayleigh_quotients_all.index(min(rayleigh_quotients_all))]
min_ind_vector = np.zeros((len(adj_matrix),), dtype=np.complex128)

for i, cluster in enumerate(partition_numbered):
    min_ind_vector[cluster] = np.exp(2 * np.pi * 1j * min_perm[i] / k)

min_ind_vector = D_sqrt @ min_ind_vector
min_ind_vector = min_ind_vector / np.linalg.norm(min_ind_vector)

# compute Psi
Psis = []
for perm in itertools.permutations(range(k)):
    C = [(perm[i], perm[i + 1]) for i in range(k - 1)] + [(perm[k - 1], perm[0])]
    Psis.append(Psi(partition_numbered, adj_matrix, degrees, C))

min_perm_psi = list(itertools.permutations(range(k)))[Psis.index(min(Psis))]

# rotate indicator vector to align with first eigenvector
alpha = np.conj(min_ind_vector).T @ eigvecs[:, 0]
min_ind_vector = alpha * min_ind_vector

thetas = []

for perm in itertools.permutations(range(k)):
    C = [(perm[i], perm[i + 1]) for i in range(len(perm) - 1)]
    C.append((perm[-1], perm[0]))
    theta = compute_theta(partition_numbered, adj_matrix, degrees, C)
    thetas.append(theta)
    
max_perm_theta = list(itertools.permutations(range(k)))[thetas.index(max(thetas))]
print("Max perm theta ", max_perm_theta)
C = [(max_perm_theta[i], max_perm_theta[i + 1]) for i in range(len(max_perm_theta) - 1)]
theta = compute_theta(partition_numbered, adj_matrix, degrees, C)

compute_ls_bounds(eigvals_ls, theta, k)
eigvals_ls[1] / (1 - (4 / k) * theta)
eta = eigvals_ls[1] / (1 - (4 / k) * theta)

print("Number of vertices: ", len(adj_matrix))
print("Number of Edges: ", np.sum(np.sum(adj_matrix, axis=1)))
print("Theta", theta)
print("1/eta_k", 1 / eta)
print("1/(eta_k-1)", 1 / (eta - 1))
print("Psi", min(Psis))
print("rayleigh quotient", min(rayleigh_quotients_all))
print("Our bound (Rayleigh Quotient) ",(min(rayleigh_quotients_all) - eigvals[0]) / (eigvals[1] - eigvals[0]))
print("Our bound (Psi) ", (4*min(Psis) - eigvals[0])/ (eigvals[1] - eigvals[0]))
print("True value ", np.linalg.norm(min_ind_vector - eigvecs[:, 0]) ** 2)

## St. Marks Seagrass
Using k = 5

In [ ]:

urllib.request.urlretrieve("http://cosinproject.eu/extra/data/foodwebs/seagrass.txt", "Data/seagrass.txt")
# construct adjacency matrix
with open("Data/seagrass.txt", "r") as f:
    lines = f.readlines()
    edges = [tuple(map(int, line.strip().split())) for line in lines]

edges_df = pd.DataFrame(edges, columns=["source", "target"])
edges_df_unweighted = edges_df.applymap(int)
edges_df_unweighted.head(10)
# convert edges_df_unweighted to adjacency matrix
index = list(set(edges_df_unweighted["source"].values) | set(edges_df_unweighted["target"].values))
index.sort()
adj_matrix = np.zeros((len(index), len(index)))
for i, row in edges_df_unweighted.iterrows():
    source = row["source"]
    target = row["target"]
    source_index = index.index(source)
    target_index = index.index(target)
    adj_matrix[source_index, target_index] = 1

k = 5
k_ls = int(np.ceil(2 * np.pi * k))
herm_A = np.exp(1j * 2 * np.pi / k) * adj_matrix + np.exp(-1j * 2 * np.pi / k) * adj_matrix.T
herm_A_ls = np.exp(1j * 2 * np.pi / k_ls) * adj_matrix + np.exp(-1j * 2 * np.pi / k_ls) * adj_matrix.T
degrees = np.sum(np.abs(herm_A), axis=1)
D_inv_sqrt = np.diag(1 / np.sqrt(degrees))
D_sqrt = np.diag(np.sqrt(degrees))
norm_L = np.eye(len(adj_matrix)) - D_inv_sqrt @ herm_A @ D_inv_sqrt
norm_L_ls = np.eye(len(adj_matrix)) - D_inv_sqrt @ herm_A_ls @ D_inv_sqrt

eigvals, eigvecs = np.linalg.eigh(norm_L)
idx = eigvals.argsort()
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

eigvals_ls, eigvecs_ls = np.linalg.eigh(norm_L_ls)
idx = eigvals_ls.argsort()
eigvals_ls = eigvals_ls[idx]
eigvecs_ls = eigvecs_ls[:, idx]

# cluster using first eigenvector
datapoints = np.column_stack((D_inv_sqrt @ eigvecs[:, 0].real, D_inv_sqrt @ eigvecs[:, 0].imag))
kmeans = KMeans(n_clusters=k, random_state=0).fit(datapoints)
partition_numbered = [np.where(kmeans.labels_ == i)[0] for i in range(k)]

rayleigh_quotients_all = []
for perm in itertools.permutations(range(k)):
    # construct indicator vector of clusters
    ind_vector = np.zeros((len(adj_matrix),), dtype=np.complex128)

    for i, cluster in enumerate(partition_numbered):
        ind_vector[cluster] = np.exp(2 * np.pi * 1j * perm[i] / k)

    ind_vector = D_sqrt @ ind_vector
    ind_vector = ind_vector / np.linalg.norm(ind_vector)

    # rotate indicator vector to align with first eigenvector
    alpha = np.conj(ind_vector).T @ eigvecs[:, 0]
    ind_vector = alpha * ind_vector
    ind_vector = ind_vector / np.linalg.norm(ind_vector)

    rayleigh_quotients_all.append(np.real(np.conj(ind_vector).T @ norm_L @ ind_vector))


# set indicator vector to be the one with the minimum rayleigh quotient
min_perm = list(itertools.permutations(range(k)))[rayleigh_quotients_all.index(min(rayleigh_quotients_all))]
min_ind_vector = np.zeros((len(adj_matrix),), dtype=np.complex128)

for i, cluster in enumerate(partition_numbered):
    min_ind_vector[cluster] = np.exp(2 * np.pi * 1j * min_perm[i] / k)

min_ind_vector = D_sqrt @ min_ind_vector
min_ind_vector = min_ind_vector / np.linalg.norm(min_ind_vector)

# compute Psi
Psis = []
for perm in itertools.permutations(range(k)):
    C = [(perm[i], perm[i + 1]) for i in range(k - 1)] + [(perm[k - 1], perm[0])]
    Psis.append(Psi(partition_numbered, adj_matrix, degrees, C))

min_perm_psi = list(itertools.permutations(range(k)))[Psis.index(min(Psis))]

# rotate indicator vector to align with first eigenvector
alpha = np.conj(min_ind_vector).T @ eigvecs[:, 0]
min_ind_vector = alpha * min_ind_vector

thetas = []

for perm in itertools.permutations(range(k)):
    C = [(perm[i], perm[i + 1]) for i in range(len(perm) - 1)]
    C.append((perm[-1], perm[0]))
    theta = compute_theta(partition_numbered, adj_matrix, degrees, C)
    thetas.append(theta)
    
max_perm_theta = list(itertools.permutations(range(k)))[thetas.index(max(thetas))]
print("Max perm theta ", max_perm_theta)
C = [(max_perm_theta[i], max_perm_theta[i + 1]) for i in range(len(max_perm_theta) - 1)]
theta = compute_theta(partition_numbered, adj_matrix, degrees, C)

compute_ls_bounds(eigvals_ls, theta, k)
eigvals_ls[1] / (1 - (4 / k) * theta)
eta = eigvals_ls[1] / (1 - (4 / k) * theta)
print("Number of vertices: ", len(adj_matrix))
print("Number of Edges: ", np.sum(np.sum(adj_matrix, axis=1)))
print("Theta", theta)
print("1/eta_k", 1 / eta)
print("1/(eta_k-1)", 1 / (eta - 1))
print("Psi", min(Psis))
print("rayleigh quotient", min(rayleigh_quotients_all))
print("Our bound (Rayleigh Quotient) ",(min(rayleigh_quotients_all) - eigvals[0]) / (eigvals[1] - eigvals[0]))
print("Our bound (Psi) ", (4*min(Psis) - eigvals[0])/ (eigvals[1] - eigvals[0]))
print("True value ", np.linalg.norm(min_ind_vector - eigvecs[:, 0]) ** 2)